In [40]:
#Bibliotecas e Variaveis Uteis
#!pip install requirements.txt -r
import geopandas as gpd
from calendar import monthrange
from collections import defaultdict
import shutil
import matplotlib.pyplot as plt
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed
from scipy import stats
from scipy.signal import find_peaks
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import gc
import pandas as pd
import numpy as np
import xarray as xr
import pyarrow.parquet as pa_parquet
import cdsapi, backoff, zipfile, shutil, time
import os, json
import time
from glob import glob
from tqdm import tqdm
import netCDF4
import h5netcdf
import requests
import fastparquet
import dask.dataframe as dd
import unicodedata
from pathlib import Path
import polars as pl
import hvplot.pandas
from polars import selectors as cs
from shapely.geometry import Point
import seaborn as sns

pasta_dados = r'C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados'

# 2. Lista de todas as pastas que precisam existir
# Extraímos apenas o diretório de cada caminho de arquivo definido
pastas_necessarias = [
    pasta_dados,
    os.path.join(pasta_dados, 'Series_Temporais'),
    os.path.join(pasta_dados, 'ERA5_Bruto'),
    os.path.join(pasta_dados, 'EDA'),
    os.path.join(pasta_dados, 'EDA', 'visualizacoes'),
    os.path.join(pasta_dados, 'EDA', 'consolidados'),
    os.path.join(pasta_dados, 'Potencia_ONS'),
]

# 3. Loop para criar as pastas automaticamente
print("Verificando e criando diretórios...")
for pasta in pastas_necessarias:
    os.makedirs(pasta, exist_ok=True)
    print(f"OK: {pasta}")

#Caminhos ruins
caminho_series_temporais = os.path.join(pasta_dados, 'Series_Temporais', 'series_temporais_bruto.parquet')
caminho_dados_estaticos = os.path.join(pasta_dados, 'Dataset_Locais.csv')
caminho_dataset_final = os.path.join(pasta_dados, 'Dataset_Final_Para_Modelagem.parquet')
caminho_dataset_intermediario = os.path.join(pasta_dados, 'Dataset_Intermediario.parquet')
output_path = os.path.join(pasta_dados, 'Series_Temporais')

#Limpo
pasta_era5 = os.path.join(pasta_dados, 'ERA5_Bruto')
serie_temporal = os.path.join(pasta_dados, 'Series_Temporais')
tratado_era5 = os.path.join(serie_temporal,  'series_temporais_bruto.parquet')
dataset_locais = os.path.join(pasta_dados, 'Dataset_Locais.csv')
#dataset_locais = os.path.join(pasta_dados, 'dataset_locais_semNaN.csv')
pot_ons = os.path.join(pasta_dados, 'Potencia_ONS')

pq = pasta_dados + "//Series_Temporais//series_temporais_*.parquet"
PARQUETS = glob(pq)

#Limpeza e Trat dos dados
EDA_DIR = os.path.join(pasta_dados, 'EDA') #"/content/drive/MyDrive/PUC/IC/ColetaEra5/Dados/EDA"
os.makedirs(EDA_DIR, exist_ok=True)
viz_dir = os.path.join(EDA_DIR, "visualizacoes")
os.makedirs(viz_dir, exist_ok=True)

percent_glob = os.path.join(EDA_DIR, "percentis_globais.csv")
consolidado = os.path.join(EDA_DIR, "consolidados")
global_stat = os.path.join(consolidado,"estatisticas_globais.csv")
hist = os.path.join(consolidado, "histogramas_globais.csv")
valid = os.path.join(consolidado, "validacoes_detalhadas.csv")





#Variaveis Uteis

# Configuração de download
anos = range(2015, 2026); meses = [str(m).zfill(2) for m in range(1, 13)]
#anos = [2024]; meses = ['06']

subsistema_nordeste_ufs = [
    'MA',  # Maranhão
    'PI',  # Piauí
    'CE',  # Ceará
    'RN',  # Rio Grande do Norte
    'PB',  # Paraíba
    'PE',  # Pernambuco
    'AL',  # Alagoas
    'SE',  # Sergipe
    'BA'   # Bahia
]

status_operacional = [
    'Operação', 'Construção não iniciada', 'Construção', 'DRO'
    ]
#DRO -> Despacho de Requerimento de Outorga -> Entrara no longo prazo (talvez mais de 5 anos)
#Construção não iniciada -> Entrara no médio prazo (menos de 5 anos)
#Construção -> Entrara no curto prazo (de 1-3 anos)


Verificando e criando diretórios...
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\Series_Temporais
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\ERA5_Bruto
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\visualizacoes
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\consolidados
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\Potencia_ONS


## Coleta de Dados

In [41]:
##Coleta ONS
#Fato de Capacidade fora da analise.


BASE_S3 = "https://ons-aws-prod-opendata.s3.amazonaws.com/dataset"
OUT_DIR = pot_ons
os.makedirs(os.path.dirname(OUT_DIR), exist_ok=True)

def downloadFile(anos, alvo, url_spec):

    for y in anos:
        name = f"{alvo}_{y}.csv"
        url = f"{BASE_S3}/{url_spec}/{alvo}_{y}.csv"
        out_path = os.path.join(OUT_DIR, name)
        
        
        if os.path.exists(out_path):
            print(f"Arquivo {name} existe")
            continue
        
        
        with requests.get(url, stream=True, timeout=120) as r:
            if r.status_code == 404:
                return "not_found"
            r.raise_for_status()

            tmp = out_path + ".part"
            with open(tmp, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024 * 1024):
                    if chunk:
                        f.write(chunk)
            os.replace(tmp, out_path)
        



lst_alvos = [["BALANCO_ENERGIA_SUBSISTEMA", 'balanco_energia_subsistema_ho' ],[ "CARGA_ENERGIA", 'carga_energia_di'], ["CURVA_CARGA", 'curva-carga-ho']]




max_workers = 6
results = {"ok": 0, "skip": 0, "not_found": 0, "fail": 0}

with ThreadPoolExecutor(max_workers=max_workers) as ex:
    futures = {}
    for alvo in lst_alvos:
        downloadFile(anos, alvo[0], alvo[1])

print("\nResumo download:", results)
print("Pasta cache ONS:", os.path.abspath(OUT_DIR))


Arquivo BALANCO_ENERGIA_SUBSISTEMA_2015.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2016.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2017.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2018.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2019.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2020.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2021.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2022.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2023.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2024.csv existe
Arquivo BALANCO_ENERGIA_SUBSISTEMA_2025.csv existe
Arquivo CARGA_ENERGIA_2015.csv existe
Arquivo CARGA_ENERGIA_2016.csv existe
Arquivo CARGA_ENERGIA_2017.csv existe
Arquivo CARGA_ENERGIA_2018.csv existe
Arquivo CARGA_ENERGIA_2019.csv existe
Arquivo CARGA_ENERGIA_2020.csv existe
Arquivo CARGA_ENERGIA_2021.csv existe
Arquivo CARGA_ENERGIA_2022.csv existe
Arquivo CARGA_ENERGIA_2023.csv existe
Arquivo CARGA_ENERGIA_2024.csv existe
Arquivo CARGA_ENERGIA_2025.csv existe
Arquivo CURVA_CARGA_2

In [42]:
## Coleta ERA5 - Médias Mensais por Célula do Grid

import pandas as pd
import numpy as np
import cdsapi
import os
from glob import glob
from tqdm import tqdm
import time
import backoff

print("="*100)
print("COLETA ERA5 - DADOS METEOROLOGICOS MENSAIS")
print("="*100)

# ==============================================================================
# CONSTANTES E VARIÁVEIS
# ==============================================================================
RES = 0.25  # Resolução do grid ERA5 (0.25°)

# Variáveis a coletar (mesmas do código original)
vars = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "100m_u_component_of_wind",
    "100m_v_component_of_wind",
    "2m_temperature",
    "sea_surface_temperature",
    "surface_pressure",
    "total_precipitation",
    "friction_velocity"
]

# CORREÇÃO: Adicionar definição de meses que estava faltando
meses = [str(m).zfill(2) for m in range(1, 13)]  # ['01', '02', ..., '12']

lst_errors = []

# ==============================================================================
# FUNÇÕES UTILITÁRIAS (COPIADAS DO ORIGINAL)
# ==============================================================================

def snap_to_era5_grid(lat, lon, res=RES):
    """Arredonda lat/lon para o grid ERA5 mais próximo"""
    lat_era5 = round(lat / res) * res
    lon_era5 = round(lon / res) * res
    return lat_era5, lon_era5

def criar_cell_key(lat_era5, lon_era5):
    """Cria identificador único para célula do grid"""
    lat_str = f"latm{abs(lat_era5):.2f}".replace(".", "p")
    lon_str = f"lonm{abs(lon_era5):.2f}".replace(".", "p")
    return f"{lat_str}_{lon_str}"

def bbox_single_cell(lat_era5: float, lon_era5: float, res: float = RES):
    """Cria bounding box de uma célula do grid"""
    half_res = res / 2.0
    norte = lat_era5 + half_res
    sul = lat_era5 - half_res
    oeste = lon_era5 - half_res
    leste = lon_era5 + half_res
    return [norte, oeste, sul, leste]

def parse_cell_key(cell_key: str):
    """Extrai lat/lon de um cell_key"""
    # Ex: "latm10p00_lonm38p75" -> (-10.00, -38.75)
    parts = cell_key.split("_")
    lat_str = parts[0].replace("latm", "-").replace("p", ".")
    lon_str = parts[1].replace("lonm", "-").replace("p", ".")
    return float(lat_str), float(lon_str)

# ==============================================================================
# FUNÇÃO DE DOWNLOAD (COPIADA DO ORIGINAL)
# ==============================================================================

# Cliente CDS API
c = cdsapi.Client()

def retrieve_year_monthly_means(ano: int, bbox: list, output_file: str):
    """Baixa médias mensais do ERA5 para um ano e uma bbox"""
    c.retrieve(
        "reanalysis-era5-single-levels-monthly-means",
        {
            "product_type": "monthly_averaged_reanalysis",
            "variable": vars,
            "year": str(ano),
            "month": meses,  # ['01', '02', ..., '12']
            "time": "00:00",
            "area": bbox,
            "format": "netcdf",
            "grid": "0.25/0.25",
        },
        output_file
    )

# Função para tratar erros e fazer retry
def sleep_por_erro(e: Exception):
    """Define tempo de espera baseado no erro"""
    msg = str(e).lower()

    if "429" in msg or "too many requests" in msg:
        return 300  # 5 minutos
    elif "503" in msg or "service unavailable" in msg:
        return 180  # 3 minutos
    elif "connection" in msg or "timeout" in msg:
        return 60   # 1 minuto
    else:
        return 30   # 30 segundos

@backoff.on_exception(
    backoff.expo,
    Exception,
    max_tries=5,
    giveup=lambda e: "404" in str(e).lower()
)
def download_com_retry(ano, bbox, output_file):
    """Download com retry automático"""
    try:
        retrieve_year_monthly_means(ano, bbox, output_file)
        return True
    except Exception as e:
        sleep_time = sleep_por_erro(e)
        print(f"   ERRO: {e}")
        print(f"   Aguardando {sleep_time}s antes de retry...")
        time.sleep(sleep_time)
        raise

# ==============================================================================
# 1. CARREGAR E PREPARAR DADOS DOS PARQUES
# ==============================================================================
print("\n1. Carregando dados dos parques...")

df_locais = pd.read_csv(dataset_locais)
print(f"   Total de registros: {len(df_locais)}")

# Normalizar CEG (remover pontos, último ponto vira hífen)
def normalizar_ceg(ceg):
    """Normaliza CEG para buscar em CAPACIDADE_GERACAO"""
    if pd.isna(ceg):
        return None
    s = str(ceg).strip().upper()
    if s in {"", "-"}:
        return None
    
    last_dot_idx = s.rfind('.')
    if last_dot_idx != -1:
        s = s[:last_dot_idx] + '-' + s[last_dot_idx+1:]
    
    s = s.replace(".", "")
    s = s.replace(" ", "")
    return s

df_locais['CEG_norm'] = df_locais['CEG'].apply(normalizar_ceg)

# Filtrar apenas NE
df_locais_ne = df_locais[df_locais['uf'].isin(subsistema_nordeste_ufs)].copy()
print(f"   Registros no NE: {len(df_locais_ne)}")

# Agrupar por parque (CEG único, coordenadas do parque)
df_parques = df_locais_ne.groupby(
    ['CEG_norm', 'nome_parque', 'longitude_parque', 'latitude_parque', 'uf'],
    as_index=False
).agg({
    'qtd_aerogeradores': 'first',
    'potencia_turbina_mw': 'sum',  # Soma das turbinas do parque
})

print(f"   Parques únicos no NE: {len(df_parques)}")

# ==============================================================================
# 2. BUSCAR CAPACIDADE EM CAPACIDADE_GERACAO.CSV
# ==============================================================================
print("\n2. Buscando capacidade instalada em CAPACIDADE_GERACAO.csv...")

arquivo_cap = os.path.join(pot_ons, "CAPACIDADE_GERACAO.csv")
df_cap = pd.read_csv(arquivo_cap, sep=';')
print(f"   Total de registros: {len(df_cap)}")

# Filtrar apenas eólicas no NE
df_cap_eol_ne = df_cap[
    (df_cap['nom_tipousina'].str.upper().str.contains('EOLI', na=False)) &
    (df_cap['nom_subsistema'].str.upper().str.contains('NORDESTE', na=False))
].copy()

print(f"   Registros eólicos no NE: {len(df_cap_eol_ne)}")

# Normalizar CEG
df_cap_eol_ne['ceg_norm'] = df_cap_eol_ne['ceg'].apply(normalizar_ceg)

# Converter potência
df_cap_eol_ne['val_potenciaefetiva'] = pd.to_numeric(
    df_cap_eol_ne['val_potenciaefetiva'], errors='coerce'
)

# Agregar capacidade por CEG
capacidade_por_ceg = df_cap_eol_ne.groupby('ceg_norm')['val_potenciaefetiva'].sum()

print(f"   CEGs únicos com capacidade: {len(capacidade_por_ceg)}")

# Merge com df_parques
df_parques['capacidade_mw'] = df_parques['CEG_norm'].map(capacidade_por_ceg)

# CORREÇÃO: Usar fillna sem inplace para evitar FutureWarning
df_parques['capacidade_mw'] = df_parques['capacidade_mw'].fillna(df_parques['potencia_turbina_mw'])

print(f"   Capacidade total NE: {df_parques['capacidade_mw'].sum():.2f} MW")
print(f"   Parques sem capacidade: {df_parques['capacidade_mw'].isna().sum()}")

# Remover parques sem capacidade
df_parques = df_parques[df_parques['capacidade_mw'].notna()].copy()

print(f"   Parques finais: {len(df_parques)}")

# ==============================================================================
# 3. CRIAR CELL_KEYS (CÉLULAS DO GRID ERA5)
# ==============================================================================
print("\n3. Criando cell_keys (células do grid ERA5)...")

# Snap para grid ERA5
df_parques[['lat_era5', 'lon_era5']] = df_parques.apply(
    lambda r: pd.Series(snap_to_era5_grid(r['latitude_parque'], r['longitude_parque'])),
    axis=1
)

# Criar cell_key
df_parques['cell_key'] = df_parques.apply(
    lambda r: criar_cell_key(r['lat_era5'], r['lon_era5']),
    axis=1
)

print(f"   Cell_keys únicos: {df_parques['cell_key'].nunique()}")

# DataFrame de células únicas
df_cells = df_parques[['cell_key', 'lat_era5', 'lon_era5']].drop_duplicates().reset_index(drop=True)

print(f"   Células a baixar: {len(df_cells)}")
print(f"\n   Primeiras 5 células:")
print(df_cells.head().to_string())

# Salvar mapeamento parque -> cell_key para uso posterior
mapeamento_file = os.path.join(pasta_dados, 'mapeamento_parques_cells.csv')
df_parques.to_csv(mapeamento_file, index=False)
print(f"\n   Mapeamento salvo: {mapeamento_file}")

# ==============================================================================
# 4. DOWNLOAD DOS DADOS ERA5
# ==============================================================================
print("\n4. Baixando dados ERA5...")

# Para cada célula, baixar dados de todos os anos
total_downloads = len(df_cells) * len(list(anos))
downloads_ok = 0
downloads_skip = 0
downloads_erro = 0

for _, cell_row in tqdm(df_cells.iterrows(), total=len(df_cells), desc="Células"):
    cell_key = cell_row['cell_key']
    lat_era5 = cell_row['lat_era5']
    lon_era5 = cell_row['lon_era5']

    bbox = bbox_single_cell(lat_era5, lon_era5)

    for ano in anos:
        # Criar diretório para o ano
        ano_dir = os.path.join(pasta_era5, f"ano={ano}")
        os.makedirs(ano_dir, exist_ok=True)

        # Nome do arquivo
        output_file = os.path.join(ano_dir, f"Era5M_NE-{cell_key}_{ano}.nc")

        # Verificar se já existe
        if os.path.exists(output_file):
            downloads_skip += 1
            continue

        # Download
        try:
            download_com_retry(ano, bbox, output_file)
            downloads_ok += 1
            print(f"   OK: {cell_key} - {ano}")
            time.sleep(1)  # Pausa para não sobrecarregar API

        except Exception as e:
            downloads_erro += 1
            lst_errors.append({
                'cell_key': cell_key,
                'ano': ano,
                'erro': str(e)
            })
            print(f"   ERRO: {cell_key} - {ano}: {e}")

# ==============================================================================
# 5. RESUMO
# ==============================================================================
print("\n" + "="*100)
print("RESUMO DO DOWNLOAD")
print("="*100)
print(f"\nTotal esperado: {total_downloads}")
print(f"  - Downloads OK: {downloads_ok}")
print(f"  - Já existiam: {downloads_skip}")
print(f"  - Erros: {downloads_erro}")

if lst_errors:
    print(f"\n⚠️  {len(lst_errors)} downloads falharam:")
    df_errors = pd.DataFrame(lst_errors)
    print(df_errors.to_string())

    # Salvar erros
    error_file = os.path.join(pasta_dados, 'erros_download_era5.csv')
    df_errors.to_csv(error_file, index=False)
    print(f"\n   Erros salvos em: {error_file}")

print(f"\nArquivos salvos em: {pasta_era5}/ano=YYYY/")
print(f"Próximo passo: Executar célula de Tratamento ERA5")
print("="*100)




COLETA ERA5 - DADOS METEOROLOGICOS MENSAIS

1. Carregando dados dos parques...
   Total de registros: 17758
   Registros no NE: 17758
   Parques únicos no NE: 2422

2. Buscando capacidade instalada em CAPACIDADE_GERACAO.csv...
   Total de registros: 5464
   Registros eólicos no NE: 1938
   CEGs únicos com capacidade: 946
   Capacidade total NE: 62060.86 MW
   Parques sem capacidade: 0
   Parques finais: 2422

3. Criando cell_keys (células do grid ERA5)...
   Cell_keys únicos: 225
   Células a baixar: 225

   Primeiras 5 células:
              cell_key  lat_era5  lon_era5
0   latm9p00_lonm37p75     -9.00    -37.75
1   latm9p25_lonm39p00     -9.25    -39.00
2   latm9p00_lonm39p00     -9.00    -39.00
3   latm9p00_lonm41p50     -9.00    -41.50
4  latm12p25_lonm42p25    -12.25    -42.25

   Mapeamento salvo: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\mapeamento_parques_cells.csv

4. Baixando dados ERA5...


Células: 100%|██████████| 225/225 [00:01<00:00, 201.04it/s]


RESUMO DO DOWNLOAD

Total esperado: 2475
  - Downloads OK: 0
  - Já existiam: 2475
  - Erros: 0

Arquivos salvos em: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\ERA5_Bruto/ano=YYYY/
Próximo passo: Executar célula de Tratamento ERA5


## Tratamento de Dados e Geração de Estatisticas

In [43]:
## Processamento ONS

print("="*100)
print("UNIFICACAO DOS DADOS ONS - DADOS MENSAIS AGREGADOS PARA O NE")
print("="*100)

# Diretorio base
data_dir = pot_ons

# Anos disponiveis
anos_range = list(range(2015, 2026))  # 2015 a 2025

# ==============================================================================
# 1. PROCESSAR BALANCO_ENERGIA_SUBSISTEMA - Geracao eolica mensal NE
# ==============================================================================
print("\n1. Processando BALANCO_ENERGIA_SUBSISTEMA...")
balanco_frames = []

for ano in anos_range:
    arquivo = os.path.join(data_dir, f"BALANCO_ENERGIA_SUBSISTEMA_{ano}.csv")
    if not os.path.exists(arquivo):
        print(f"   [AVISO] Arquivo nao encontrado: {arquivo}")
        continue

    print(f"   Lendo: {ano}...")
    df = pd.read_csv(arquivo, sep=';')

    # Filtrar apenas NORDESTE
    df_ne = df[df['nom_subsistema'].str.upper().str.contains('NORDESTE', na=False)].copy()

    # Converter data
    df_ne['din_instante'] = pd.to_datetime(df_ne['din_instante'], errors='coerce')
    df_ne['ano'] = df_ne['din_instante'].dt.year
    df_ne['mes'] = df_ne['din_instante'].dt.month

    # Converter valores
    df_ne['val_gereolica'] = pd.to_numeric(df_ne['val_gereolica'], errors='coerce')
    df_ne['val_carga'] = pd.to_numeric(df_ne['val_carga'], errors='coerce')

    # Agregar por mes
    df_mensal = df_ne.groupby(['ano', 'mes'], as_index=False).agg({
        'val_gereolica': 'mean',  # Media mensal em MWmed
        'val_carga': 'mean'       # Carga media mensal do NE em MWmed
    })

    df_mensal.rename(columns={
        'val_gereolica': 'geracao_eolica_ne_mwmed',
        'val_carga': 'carga_ne_mwmed'
    }, inplace=True)

    balanco_frames.append(df_mensal)
    print(f"      -> {len(df_mensal)} meses processados")

df_balanco = pd.concat(balanco_frames, ignore_index=True)
print(f"\n   Total: {len(df_balanco)} registros mensais (geracao eolica + carga NE)")

# ==============================================================================
# 2. PROCESSAR CAPACIDADE_GERACAO - Capacidade instalada eolica NE (COM DESATIVAÇÃO)
# ==============================================================================
print("\n2. Processando CAPACIDADE_GERACAO (considerando entrada E desativação)...")
arquivo_cap = os.path.join(data_dir, "CAPACIDADE_GERACAO.csv")

df_cap = pd.read_csv(arquivo_cap, sep=';')
print(f"   Total de registros: {len(df_cap)}")

# Filtrar apenas EOLICA no NORDESTE
df_cap_eol_ne = df_cap[
    (df_cap['nom_tipousina'].str.upper().str.contains('EOLI', na=False)) &
    (df_cap['nom_subsistema'].str.upper().str.contains('NORDESTE', na=False))
].copy()

print(f"   Registros eolicos no NE: {len(df_cap_eol_ne)}")

# Converter datas de ENTRADA e DESATIVAÇÃO
df_cap_eol_ne['dat_entradaoperacao'] = pd.to_datetime(df_cap_eol_ne['dat_entradaoperacao'], errors='coerce')
df_cap_eol_ne['dat_desativacao'] = pd.to_datetime(df_cap_eol_ne['dat_desativacao'], errors='coerce')

# Converter potencia
df_cap_eol_ne['val_potenciaefetiva'] = pd.to_numeric(df_cap_eol_ne['val_potenciaefetiva'], errors='coerce')

# Remover registros sem data de entrada ou potencia
df_cap_eol_ne = df_cap_eol_ne.dropna(subset=['dat_entradaoperacao', 'val_potenciaefetiva'])

print(f"   Registros com data de entrada: {len(df_cap_eol_ne)}")
print(f"   Registros com data de desativação: {df_cap_eol_ne['dat_desativacao'].notna().sum()}")

# Criar um DataFrame com todos os meses de 2015 a 2025
todos_meses = []
for ano in range(2015, 2026):
    for mes in range(1, 13):
        todos_meses.append({'ano': ano, 'mes': mes})
df_meses = pd.DataFrame(todos_meses)

# Para cada mes, calcular capacidade de usinas ATIVAS
capacidades = []
for _, row in df_meses.iterrows():
    ano_ref = row['ano']
    mes_ref = row['mes']
    
    # Data de referência do mês
    data_ref = pd.Timestamp(year=ano_ref, month=mes_ref, day=1)
    
    # Filtro: usina ATIVA neste mês
    # Ativa = data_ref >= dat_entradaoperacao AND (dat_desativacao is NULL OR data_ref < dat_desativacao)
    mask = (
        (df_cap_eol_ne['dat_entradaoperacao'] <= data_ref) &
        (
            df_cap_eol_ne['dat_desativacao'].isna() |
            (df_cap_eol_ne['dat_desativacao'] > data_ref)
        )
    )
    
    cap_ativa = df_cap_eol_ne[mask]['val_potenciaefetiva'].sum()
    capacidades.append(cap_ativa)

df_meses['capacidade_eolica_ne_mw'] = capacidades

print(f"   Capacidade calculada para {len(df_meses)} meses")
print(f"   Capacidade em 2015-01: {df_meses[(df_meses['ano']==2015) & (df_meses['mes']==1)]['capacidade_eolica_ne_mw'].values[0]:.2f} MW")
print(f"   Capacidade em 2025-12: {df_meses[(df_meses['ano']==2025) & (df_meses['mes']==12)]['capacidade_eolica_ne_mw'].values[0]:.2f} MW")

# ==============================================================================
# 3. PROCESSAR CARGA_ENERGIA - Demanda mensal SIN completo
# ==============================================================================
print("\n3. Processando CARGA_ENERGIA (SIN completo)...")
carga_frames = []

for ano in anos_range:
    arquivo = os.path.join(data_dir, f"CARGA_ENERGIA_{ano}.csv")
    if not os.path.exists(arquivo):
        print(f"   [AVISO] Arquivo nao encontrado: {arquivo}")
        continue

    print(f"   Lendo: {ano}...")
    df = pd.read_csv(arquivo, sep=';')

    # Converter data
    df['din_instante'] = pd.to_datetime(df['din_instante'], errors='coerce')
    df['ano'] = df['din_instante'].dt.year
    df['mes'] = df['din_instante'].dt.month

    # Converter valor
    df['val_cargaenergiamwmed'] = pd.to_numeric(df['val_cargaenergiamwmed'], errors='coerce')

    # Agregar por dia (soma de todos subsistemas)
    df_dia = df.groupby(['ano', 'mes', 'din_instante'], as_index=False).agg({
        'val_cargaenergiamwmed': 'sum'
    })

    # Agregar por mes (media mensal)
    df_mensal = df_dia.groupby(['ano', 'mes'], as_index=False).agg({
        'val_cargaenergiamwmed': 'mean'
    })

    df_mensal.rename(columns={'val_cargaenergiamwmed': 'demanda_sin_mwmed'}, inplace=True)
    carga_frames.append(df_mensal)
    print(f"      -> {len(df_mensal)} meses processados")

df_carga = pd.concat(carga_frames, ignore_index=True)
print(f"\n   Total: {len(df_carga)} registros mensais (demanda SIN)")

# ==============================================================================
# 4. PROCESSAR CURVA_CARGA - Demanda maxima mensal SIN
# ==============================================================================
print("\n4. Processando CURVA_CARGA...")
curva_frames = []

for ano in anos_range:
    arquivo = os.path.join(data_dir, f"CURVA_CARGA_{ano}.csv")
    if not os.path.exists(arquivo):
        print(f"   [AVISO] Arquivo nao encontrado: {arquivo}")
        continue

    print(f"   Lendo: {ano}...")
    df = pd.read_csv(arquivo, sep=';')

    # Converter data
    df['din_instante'] = pd.to_datetime(df['din_instante'], errors='coerce')
    df['ano'] = df['din_instante'].dt.year
    df['mes'] = df['din_instante'].dt.month

    # Converter valor
    df['val_cargaenergiahomwmed'] = pd.to_numeric(df['val_cargaenergiahomwmed'], errors='coerce')

    # Para cada hora, somar todos os subsistemas (SIN total)
    df_hora = df.groupby(['ano', 'mes', 'din_instante'], as_index=False).agg({
        'val_cargaenergiahomwmed': 'sum'
    })

    # Para cada mes, pegar o maximo (pico de demanda)
    df_mensal = df_hora.groupby(['ano', 'mes'], as_index=False).agg({
        'val_cargaenergiahomwmed': 'max'
    })

    df_mensal.rename(columns={'val_cargaenergiahomwmed': 'demanda_max_sin_mw'}, inplace=True)
    curva_frames.append(df_mensal)
    print(f"      -> {len(df_mensal)} meses processados")

df_curva = pd.concat(curva_frames, ignore_index=True)
print(f"\n   Total: {len(df_curva)} registros mensais (demanda maxima SIN)")

# ==============================================================================
# 5. UNIFICAR TUDO
# ==============================================================================
print("\n5. Unificando dados...")

# Comecar com balanco (geracao eolica + carga NE)
df_final = df_balanco.copy()

# Merge com capacidade
df_final = df_final.merge(df_meses[['ano', 'mes', 'capacidade_eolica_ne_mw']],
                           on=['ano', 'mes'], how='left')

# Merge com carga SIN
df_final = df_final.merge(df_carga, on=['ano', 'mes'], how='left')

# Merge com curva (demanda maxima)
df_final = df_final.merge(df_curva, on=['ano', 'mes'], how='left')

# Adicionar coluna de subsistema
df_final['subsistema'] = 'NE'

# ==============================================================================
# 6. FEATURES DERIVADAS
# ==============================================================================
print("\n6. Calculando features derivadas...")

# Corte eolico (curtailment) - quando capacidade > geracao
# Em teoria, a capacidade maxima de geracao (assumindo 100% de CF) seria capacidade_mw
# Na pratica, com vento bom, um CF de ~50% e alto. Vamos usar uma estimativa conservadora:
# Potencial maximo mensal (horas no mes * capacidade * fator de disponibilidade ~95%)

# Horas no mes
df_final['dias_no_mes'] = df_final.apply(
    lambda row: pd.Timestamp(year=int(row['ano']), month=int(row['mes']), day=1).days_in_month,
    axis=1
)
df_final['horas_no_mes'] = df_final['dias_no_mes'] * 24

# Potencial maximo teorico (MWh) = capacidade_mw * horas * 0.95 (disponibilidade)
df_final['potencial_max_mwh'] = df_final['capacidade_eolica_ne_mw'] * df_final['horas_no_mes'] * 0.95

# Geracao efetiva (MWh) = geracao_mwmed * horas
df_final['geracao_efetiva_mwh'] = df_final['geracao_eolica_ne_mwmed'] * df_final['horas_no_mes']

# Corte eolico = potencial - efetivo (quando positivo)
# Na verdade, o corte e mais complexo, mas uma proxy e:
# Se capacity_factor muito baixo quando vento e bom = corte
# Vamos usar uma abordagem simples: assumir que CF > 40% e raro sem corte
# Corte estimado (MWmed) = max(0, capacidade * 0.40 - geracao)
# Mas isso e simplificacao. Melhor e deixar o modelo aprender.

# Por enquanto, vamos calcular o "corte potencial" como:
# Se geracao < capacidade * 0.30 (muito baixo, provavelmente sem vento, nao e corte)
# Se geracao entre 30-50% da capacidade, pode ser vento normal
# Se geracao > 50%, improvavel ter corte significativo
# O corte real viria de dados de restrições operativas do ONS que nao temos

# Vamos fazer uma estimativa conservadora:
# corte_estimado_mwmed = max(0, (capacidade * 0.45) - geracao)
# Onde 0.45 e um CF "alvo" razoavel para NE

df_final['corte_eolica_ne_mwmed'] = np.maximum(
    0,
    (df_final['capacidade_eolica_ne_mw'] * 0.45) - df_final['geracao_eolica_ne_mwmed']
)

# Capacity Factor (%)
df_final['capacity_factor_ne'] = (
    df_final['geracao_eolica_ne_mwmed'] / df_final['capacidade_eolica_ne_mw'].replace(0, np.nan)
) * 100

# Penetracao eolica no NE (%)
df_final['penetracao_eolica_ne'] = (
    df_final['geracao_eolica_ne_mwmed'] / df_final['carga_ne_mwmed'].replace(0, np.nan)
) * 100

# Penetracao eolica no SIN (%)
df_final['penetracao_eolica_sin'] = (
    df_final['geracao_eolica_ne_mwmed'] / df_final['demanda_sin_mwmed'].replace(0, np.nan)
) * 100

# Crescimento de capacidade (MW adicionados no mes)
df_final = df_final.sort_values(['ano', 'mes']).reset_index(drop=True)
df_final['crescimento_capacidade_mw'] = df_final['capacidade_eolica_ne_mw'].diff().fillna(0)

# Indice sazonal (mes do ano)
df_final['mes_sazonal'] = df_final['mes']

# Trimestre
df_final['trimestre'] = ((df_final['mes'] - 1) // 3) + 1

# ==============================================================================
# 7. ORDENAR E SELECIONAR COLUNAS FINAIS
# ==============================================================================
colunas_finais = [
    'ano', 'mes', 'subsistema',
    'geracao_eolica_ne_mwmed',
    'capacidade_eolica_ne_mw',
    'carga_ne_mwmed',
    'demanda_sin_mwmed',
    'demanda_max_sin_mw',
    'corte_eolica_ne_mwmed',
    'capacity_factor_ne',
    'penetracao_eolica_ne',
    'penetracao_eolica_sin',
    'crescimento_capacidade_mw',
    'trimestre'
]

df_final = df_final[colunas_finais]
df_final = df_final.sort_values(['ano', 'mes']).reset_index(drop=True)

print(f"\nDataset final: {df_final.shape}")
print(f"\nPrimeiras 10 linhas:")
print(df_final.head(10).to_string())
print(f"\nUltimas 10 linhas:")
print(df_final.tail(10).to_string())

print(f"\nEstatisticas:")
print(df_final.describe())

print(f"\nValores nulos por coluna:")
print(df_final.isnull().sum())

# ==============================================================================
# 8. SALVAR
# ==============================================================================
output_file = os.path.join(pasta_dados, "dados_operacao_ONS.csv")
df_final.to_csv(output_file, index=False)

print(f"\n" + "="*100)
print(f"ARQUIVO SALVO: {output_file}")
print(f"Total de registros: {len(df_final)}")
print(f"Periodo: {df_final['ano'].min()}-{df_final['mes'].min():02d} ate {df_final['ano'].max()}-{df_final['mes'].max():02d}")
print(f"\n✅ CORREÇÃO APLICADA:")
print(f"  - Capacidade agora considera entrada E desativação de usinas")
print(f"  - Apenas usinas ATIVAS em cada mês são contabilizadas")
print(f"\nColunas criadas:")
for col in colunas_finais:
    print(f"  - {col}")
print("="*100)



UNIFICACAO DOS DADOS ONS - DADOS MENSAIS AGREGADOS PARA O NE

1. Processando BALANCO_ENERGIA_SUBSISTEMA...
   Lendo: 2015...
      -> 12 meses processados
   Lendo: 2016...
      -> 12 meses processados
   Lendo: 2017...
      -> 12 meses processados
   Lendo: 2018...
      -> 12 meses processados
   Lendo: 2019...
      -> 12 meses processados
   Lendo: 2020...
      -> 12 meses processados
   Lendo: 2021...
      -> 12 meses processados
   Lendo: 2022...
      -> 12 meses processados
   Lendo: 2023...
      -> 12 meses processados
   Lendo: 2024...
      -> 12 meses processados
   Lendo: 2025...
      -> 12 meses processados

   Total: 132 registros mensais (geracao eolica + carga NE)

2. Processando CAPACIDADE_GERACAO (considerando entrada E desativação)...
   Total de registros: 5464
   Registros eolicos no NE: 1938
   Registros com data de entrada: 1938
   Registros com data de desativação: 0
   Capacidade calculada para 132 meses
   Capacidade em 2015-01: 3397.43 MW
   Capacidade

In [44]:
## Processamento ERA5 - Extração de Dados por Parque
## Correções aplicadas:
## 1. Arquivos .nc são ZIPs - agora descompacta automaticamente
## 2. Coordenada temporal é 'valid_time', não 'time'
## 3. Combina os 3 datasets (avgua, avgad, avgid) normalizando timestamps
## 4. Timestamps normalizados para apenas data (sem hora)

import pandas as pd
import numpy as np
import xarray as xr
import os
import zipfile
import tempfile
import shutil
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*100)
print("PROCESSAMENTO ERA5 - EXTRACAO DE DADOS POR PARQUE")
print("="*100)

# ==============================================================================
# FUNCAO AUXILIAR: Abrir arquivo ERA5 (ZIP ou NetCDF)
# ==============================================================================
def abrir_era5_arquivo(arquivo_path):
    """
    Abre arquivo ERA5 que pode ser ZIP (com 3 NetCDFs) ou NetCDF direto.
    Normaliza timestamps para apenas data (remove hora) antes de combinar.
    Retorna um xarray Dataset combinado com todas as variáveis.
    """
    with open(arquivo_path, 'rb') as f:
        header = f.read(4)

    if header[:2] == b'PK':  # É um arquivo ZIP
        temp_dir = tempfile.mkdtemp()

        try:
            # Extrair todos os arquivos do ZIP
            with zipfile.ZipFile(arquivo_path, 'r') as z:
                z.extractall(temp_dir)

            datasets = []
            for nc_file in glob(os.path.join(temp_dir, '*.nc')):
                ds = xr.open_dataset(nc_file, engine='netcdf4')

                # Normalizar timestamps para apenas data (primeiro dia do mês, meia-noite)
                if 'valid_time' in ds.coords:
                    # Converter para pandas datetime e normalizar
                    new_times = pd.to_datetime(ds['valid_time'].values).normalize()
                    ds = ds.assign_coords(valid_time=new_times)

                datasets.append(ds)

            if not datasets:
                raise ValueError(f"Nenhum NetCDF encontrado no ZIP: {arquivo_path}")

            # Combinar datasets (agora com timestamps alinhados)
            ds_combined = xr.merge(datasets, compat='override', join='exact')

            # Carregar em memória
            ds_memory = ds_combined.load()

            # Fechar datasets originais
            for ds in datasets:
                ds.close()
            ds_combined.close()

            return ds_memory

        finally:
            # Limpar diretório temporário
            shutil.rmtree(temp_dir, ignore_errors=True)
    else:
        # É NetCDF direto
        return xr.open_dataset(arquivo_path, engine='netcdf4')


# ==============================================================================
# 1. CARREGAR MAPEAMENTO PARQUES -> CELLS E ADICIONAR DATAS DE OPERAÇÃO
# ==============================================================================
print("\n1. Carregando mapeamento parques -> células...")

mapeamento_file = os.path.join(pasta_dados, 'mapeamento_parques_cells.csv')

if not os.path.exists(mapeamento_file):
    print(f"   ERRO: Arquivo de mapeamento não encontrado: {mapeamento_file}")
    print(f"   Execute primeiro a célula de Coleta ERA5!")
    raise FileNotFoundError(mapeamento_file)

df_parques = pd.read_csv(mapeamento_file)

print(f"   Total de parques: {len(df_parques)}")
print(f"   Cell_keys únicos: {df_parques['cell_key'].nunique()}")
print(f"   Período de processamento: {min(anos)} a {max(anos)}")

# ==============================================================================
# 1.1. ADICIONAR DATAS DE OPERAÇÃO DO CAPACIDADE_GERACAO.CSV
# ==============================================================================
print("\n1.1. Adicionando datas de operação dos parques...")

# Normalizar CEG
def normalizar_ceg(ceg):
    """Normaliza CEG para buscar em CAPACIDADE_GERACAO"""
    if pd.isna(ceg):
        return None
    s = str(ceg).strip().upper()
    if s in {"", "-"}:
        return None

    last_dot_idx = s.rfind('.')
    if last_dot_idx != -1:
        s = s[:last_dot_idx] + '-' + s[last_dot_idx+1:]

    s = s.replace(".", "")
    s = s.replace(" ", "")
    return s

# Carregar CAPACIDADE_GERACAO.csv
arquivo_cap = os.path.join(pot_ons, "CAPACIDADE_GERACAO.csv")
df_cap = pd.read_csv(arquivo_cap, sep=';')

# Filtrar apenas eólicas no NE
df_cap_eol_ne = df_cap[
    (df_cap['nom_tipousina'].str.upper().str.contains('EOLI', na=False)) &
    (df_cap['nom_subsistema'].str.upper().str.contains('NORDESTE', na=False))
].copy()

# Normalizar CEG
df_cap_eol_ne['ceg_norm'] = df_cap_eol_ne['ceg'].apply(normalizar_ceg)

# Converter datas
df_cap_eol_ne['dat_entradaoperacao'] = pd.to_datetime(df_cap_eol_ne['dat_entradaoperacao'], errors='coerce')
df_cap_eol_ne['dat_desativacao'] = pd.to_datetime(df_cap_eol_ne['dat_desativacao'], errors='coerce')

# Para cada CEG, pegar a data MÍNIMA de entrada e MÁXIMA de desativação
datas_por_ceg = df_cap_eol_ne.groupby('ceg_norm').agg({
    'dat_entradaoperacao': 'min',
    'dat_desativacao': 'max'
}).reset_index()

print(f"   CEGs com datas de operação: {len(datas_por_ceg)}")

# Merge com df_parques
df_parques = df_parques.merge(datas_por_ceg, left_on='CEG_norm', right_on='ceg_norm', how='left')

print(f"\n   Parques com data de entrada: {df_parques['dat_entradaoperacao'].notna().sum()}")
print(f"   Parques com data de desativação: {df_parques['dat_desativacao'].notna().sum()}")

# ==============================================================================
# 2. VERIFICAR ARQUIVOS DISPONÍVEIS
# ==============================================================================
print("\n2. Verificando arquivos disponíveis...")

# Padrão: Dados/ERA5_Bruto/ano=YYYY/Era5M_NE-{cell_key}_{ano}.nc
arquivos_nc = []
for ano in anos:
    ano_dir = os.path.join(pasta_era5, f"ano={ano}")
    if os.path.exists(ano_dir):
        arquivos = glob(os.path.join(ano_dir, "Era5M_NE-*.nc"))
        arquivos_nc.extend(arquivos)

print(f"   Arquivos encontrados: {len(arquivos_nc)}")

if len(arquivos_nc) == 0:
    print(f"\n   AVISO: Nenhum arquivo encontrado em {pasta_era5}/ano=YYYY/")
    print(f"   Execute primeiro a célula de Coleta ERA5!")
    raise FileNotFoundError("Nenhum arquivo encontrado")

# Verificar formato (ZIP ou NetCDF)
with open(arquivos_nc[0], 'rb') as f:
    header = f.read(4)
formato = "ZIP (com NetCDFs)" if header[:2] == b'PK' else "NetCDF direto"
print(f"   Formato detectado: {formato}")

# Criar dicionário: cell_key -> {ano: arquivo}
arquivos_por_cell = {}
for arquivo in arquivos_nc:
    nome = os.path.basename(arquivo)
    # Formato: Era5M_NE-latm10p00_lonm38p75_2015.nc
    partes = nome.replace('Era5M_NE-', '').replace('.nc', '').split('_')

    # cell_key = latm10p00_lonm38p75
    cell_key = '_'.join(partes[:-1])
    ano = int(partes[-1])

    if cell_key not in arquivos_por_cell:
        arquivos_por_cell[cell_key] = {}
    arquivos_por_cell[cell_key][ano] = arquivo

print(f"   Células únicas com dados: {len(arquivos_por_cell)}")

# ==============================================================================
# 3. PROCESSAR ARQUIVOS E EXTRAIR DADOS POR PARQUE
# ==============================================================================
print("\n3. Extraindo dados meteorológicos por parque...")

dados_extraidos = []
erros_processamento = []

# Agrupar parques por cell_key para processar por célula
parques_por_cell = df_parques.groupby('cell_key')

for cell_key, grupo_parques in tqdm(parques_por_cell, desc="Processando células"):

    if cell_key not in arquivos_por_cell:
        continue

    arquivos_ano = arquivos_por_cell[cell_key]

    # Para cada ano
    for ano in anos:
        if ano not in arquivos_ano:
            continue

        arquivo_nc = arquivos_ano[ano]

        try:
            # Abrir arquivo (ZIP ou NetCDF)
            ds = abrir_era5_arquivo(arquivo_nc)

            # Detectar nome da coordenada temporal
            time_coord = None
            for possible_name in ['valid_time', 'time', 'month']:
                if possible_name in ds.coords:
                    time_coord = possible_name
                    break

            if time_coord is None:
                raise ValueError(f"Coordenada temporal não encontrada. Coords: {list(ds.coords)}")

            # Para cada parque nesta célula
            for _, parque in grupo_parques.iterrows():

                # Coordenadas ERA5 da célula
                lat_era5 = parque['lat_era5']
                lon_era5 = parque['lon_era5']

                # Selecionar ponto mais próximo no grid
                ponto = ds.sel(latitude=lat_era5, longitude=lon_era5, method='nearest')

                # Para cada mês (time dimension)
                for t_idx, tempo in enumerate(ponto[time_coord].values):

                    # Extrair timestamp
                    timestamp = pd.Timestamp(tempo)
                    mes = timestamp.month
                    ano_ref = timestamp.year

                    # Selecionar dados do mês
                    dados_mes = ponto.isel({time_coord: t_idx})

                    # Extrair componentes de vento
                    u10 = float(dados_mes['u10'].values) if 'u10' in dados_mes else np.nan
                    v10 = float(dados_mes['v10'].values) if 'v10' in dados_mes else np.nan
                    u100 = float(dados_mes['u100'].values) if 'u100' in dados_mes else np.nan
                    v100 = float(dados_mes['v100'].values) if 'v100' in dados_mes else np.nan

                    # Calcular velocidade do vento
                    vel_vento_10m = np.sqrt(u10**2 + v10**2) if not (np.isnan(u10) or np.isnan(v10)) else np.nan
                    vel_vento_100m = np.sqrt(u100**2 + v100**2) if not (np.isnan(u100) or np.isnan(v100)) else np.nan

                    # Outras variáveis
                    t2m = float(dados_mes['t2m'].values) if 't2m' in dados_mes else np.nan
                    sp = float(dados_mes['sp'].values) if 'sp' in dados_mes else np.nan
                    tp = float(dados_mes['tp'].values) if 'tp' in dados_mes else np.nan
                    sst = float(dados_mes['sst'].values) if 'sst' in dados_mes else np.nan
                    zust = float(dados_mes['zust'].values) if 'zust' in dados_mes else np.nan

                    # Densidade do ar (rho = P / (R * T))
                    R = 287.058  # J/(kg*K) - constante específica do ar
                    densidade_ar = sp / (R * t2m) if not (np.isnan(sp) or np.isnan(t2m) or t2m == 0) else np.nan

                    # Wind shear (proxy simples: ln(v100/v10) / ln(100/10))
                    if vel_vento_100m > 0 and vel_vento_10m > 0:
                        wind_shear = np.log(vel_vento_100m / vel_vento_10m) / np.log(100.0 / 10.0)
                    else:
                        wind_shear = np.nan

                    # Armazenar dados do parque
                    dados_extraidos.append({
                        'ano': ano_ref,
                        'mes': mes,
                        'nome_parque': parque['nome_parque'],
                        'CEG_norm': parque['CEG_norm'],
                        'lat': parque['latitude_parque'],
                        'lon': parque['longitude_parque'],
                        'lat_era5': lat_era5,
                        'lon_era5': lon_era5,
                        'cell_key': cell_key,
                        'peso_capacidade_mw': parque['capacidade_mw'],  # PESO para agregação!
                        'dat_entradaoperacao': parque['dat_entradaoperacao'],
                        'dat_desativacao': parque['dat_desativacao'],
                        'uf': parque['uf'],
                        'u10': u10,
                        'v10': v10,
                        'u100': u100,
                        'v100': v100,
                        'vel_vento_10m': vel_vento_10m,
                        'vel_vento_100m': vel_vento_100m,
                        't2m': t2m,
                        'sp': sp,
                        'tp': tp,
                        'sst': sst,
                        'zust': zust,
                        'densidade_ar': densidade_ar,
                        'wind_shear': wind_shear,
                    })

            ds.close()

        except Exception as e:
            erros_processamento.append({
                'cell_key': cell_key,
                'ano': ano,
                'arquivo': arquivo_nc,
                'erro': str(e)
            })
            print(f"\n   ERRO ao processar {cell_key} - {ano}: {e}")
            continue

# ==============================================================================
# 4. CRIAR DATAFRAME FINAL
# ==============================================================================
print("\n4. Criando DataFrame final...")

df_era5_parques = pd.DataFrame(dados_extraidos)

if len(df_era5_parques) == 0:
    print(f"\n   ERRO: Nenhum dado foi extraído!")
    if erros_processamento:
        print(f"\n   Erros de processamento:")
        df_erros = pd.DataFrame(erros_processamento)
        print(df_erros.to_string())
    raise ValueError("Nenhum dado extraído dos arquivos")

print(f"   Total de registros extraídos: {len(df_era5_parques)}")
print(f"   Parques únicos: {df_era5_parques['nome_parque'].nunique()}")
print(f"   Período: {df_era5_parques['ano'].min()}/{df_era5_parques['mes'].min():02d} a {df_era5_parques['ano'].max()}/{df_era5_parques['mes'].max():02d}")

# Ordenar por ano, mês, parque
df_era5_parques = df_era5_parques.sort_values(['ano', 'mes', 'nome_parque']).reset_index(drop=True)

# ==============================================================================
# 5. VERIFICAR QUALIDADE DOS DADOS
# ==============================================================================
print("\n5. Verificando qualidade dos dados...")

print(f"\n   Valores nulos por coluna:")
nulos = df_era5_parques.isnull().sum()
if nulos.sum() > 0:
    print(nulos[nulos > 0])
else:
    print("   Nenhum valor nulo!")

print(f"\n   Estatísticas das principais variáveis:")
cols_stats = ['vel_vento_100m', 'vel_vento_10m', 't2m', 'sp', 'densidade_ar', 'peso_capacidade_mw']
print(df_era5_parques[cols_stats].describe())

# ==============================================================================
# 6. SALVAR DADOS PROCESSADOS
# ==============================================================================
print("\n6. Salvando dados processados...")

output_file = os.path.join(pasta_dados, 'dados_era5_parques_ne.csv')
df_era5_parques.to_csv(output_file, index=False)

print(f"\n" + "="*100)
print("DADOS ERA5 PROCESSADOS COM SUCESSO!")
print("="*100)
print(f"\nResumo:")
print(f"  - Arquivo salvo: {output_file}")
print(f"  - Total de registros: {len(df_era5_parques)}")
print(f"  - Parques únicos: {df_era5_parques['nome_parque'].nunique()}")
print(f"  - Período: {df_era5_parques['ano'].min()}/{df_era5_parques['mes'].min():02d} a {df_era5_parques['ano'].max()}/{df_era5_parques['mes'].max():02d}")

print(f"\nColunas do dataset:")
for i, col in enumerate(df_era5_parques.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nPrimeiras 5 linhas:")
print(df_era5_parques.head().to_string())

if erros_processamento:
    print(f"\n   AVISOS: {len(erros_processamento)} arquivos com erro:")
    df_erros = pd.DataFrame(erros_processamento)
    print(df_erros.head(10).to_string())

    # Salvar erros
    error_file = os.path.join(pasta_dados, 'erros_processamento_era5.csv')
    df_erros.to_csv(error_file, index=False)
    print(f"\n   Erros salvos em: {error_file}")

print(f"\nPróximo passo: Executar célula de Agregação ERA5")
print("="*100)


PROCESSAMENTO ERA5 - EXTRACAO DE DADOS POR PARQUE

1. Carregando mapeamento parques -> células...


   Total de parques: 2422
   Cell_keys únicos: 225
   Período de processamento: 2015 a 2025

1.1. Adicionando datas de operação dos parques...
   CEGs com datas de operação: 946

   Parques com data de entrada: 927
   Parques com data de desativação: 0

2. Verificando arquivos disponíveis...
   Arquivos encontrados: 2475
   Formato detectado: ZIP (com NetCDFs)
   Células únicas com dados: 225

3. Extraindo dados meteorológicos por parque...


Processando células: 100%|██████████| 225/225 [05:04<00:00,  1.35s/it]



4. Criando DataFrame final...
   Total de registros extraídos: 319704
   Parques únicos: 2403
   Período: 2015/01 a 2025/12

5. Verificando qualidade dos dados...

   Valores nulos por coluna:
dat_entradaoperacao    197340
dat_desativacao        319704
sst                    279840
dtype: int64

   Estatísticas das principais variáveis:
       vel_vento_100m  vel_vento_10m            t2m             sp  \
count   319704.000000  319704.000000  319704.000000  319704.000000   
mean         6.035771       4.150617     298.832619   96342.897118   
std          1.864053       1.590562       1.955270    3023.935327   
min          0.713009       0.409755     290.461090   89570.250000   
25%          4.772943       3.046142     297.716949   93923.156250   
50%          5.983365       3.936842     299.144592   95763.656250   
75%          7.376560       5.175521     300.240234   99201.437500   
max         11.903348       9.911392     304.864441  101538.898438   

        densidade_ar  peso_ca

In [45]:
## Tratamento Dados ERA5 - Agregação com Média Ponderada por Capacidade

import pandas as pd
import numpy as np
import os

print("="*100)
print("AGREGACAO ERA5 - MEDIA PONDERADA POR CAPACIDADE INSTALADA (DINAMICA)")
print("="*100)

# ==============================================================================
# 1. CARREGAR DADOS ERA5 POR PARQUE
# ==============================================================================
print("\n1. Carregando dados ERA5 por parque...")

arquivo_era5 = os.path.join(pasta_dados, 'dados_era5_parques_ne.csv')

if not os.path.exists(arquivo_era5):
    print(f"   ERRO: Arquivo não encontrado: {arquivo_era5}")
    print(f"   Execute primeiro a célula de Processamento ERA5!")
    raise FileNotFoundError(arquivo_era5)

df_era5_parques = pd.read_csv(arquivo_era5)

# Converter colunas de data
df_era5_parques['dat_entradaoperacao'] = pd.to_datetime(df_era5_parques['dat_entradaoperacao'], errors='coerce')
df_era5_parques['dat_desativacao'] = pd.to_datetime(df_era5_parques['dat_desativacao'], errors='coerce')

print(f"   Total de registros: {len(df_era5_parques)}")
print(f"   Parques únicos: {df_era5_parques['nome_parque'].nunique()}")
print(f"   Período: {df_era5_parques['ano'].min()}/{df_era5_parques['mes'].min():02d} a {df_era5_parques['ano'].max()}/{df_era5_parques['mes'].max():02d}")

# ==============================================================================
# 2. FILTRAR PARQUES POR DATA DE OPERAÇÃO (PESO DINÂMICO)
# ==============================================================================
print("\n2. Aplicando filtro temporal (parques ativos por mês)...")

# Criar coluna de data de referência (ano-mes como datetime)
df_era5_parques['data_ref'] = pd.to_datetime(
    df_era5_parques['ano'].astype(str) + '-' + df_era5_parques['mes'].astype(str).str.zfill(2) + '-01'
)

# Filtro: parque estava ATIVO naquele mês?
# Ativo = data_ref >= data_entrada AND (data_desativacao is NULL OR data_ref < data_desativacao)

registros_antes = len(df_era5_parques)

df_era5_parques['ativo'] = (
    # Parque já entrou em operação?
    (df_era5_parques['data_ref'] >= df_era5_parques['dat_entradaoperacao']) &
    # Parque ainda não foi desativado?
    (
        df_era5_parques['dat_desativacao'].isna() |
        (df_era5_parques['data_ref'] < df_era5_parques['dat_desativacao'])
    )
)

# Filtrar apenas registros ativos
df_era5_ativos = df_era5_parques[df_era5_parques['ativo']].copy()

registros_depois = len(df_era5_ativos)

print(f"   Registros ANTES do filtro: {registros_antes}")
print(f"   Registros DEPOIS do filtro (ativos): {registros_depois}")
print(f"   Registros removidos (inativos): {registros_antes - registros_depois}")
print(f"   % removido: {100 * (registros_antes - registros_depois) / registros_antes:.1f}%")

# Verificar se ainda temos dados
if len(df_era5_ativos) == 0:
    print(f"\n   ERRO: Nenhum parque ativo encontrado no período!")
    print(f"\n   Verificando datas de entrada:")
    print(df_era5_parques.groupby('ano')['dat_entradaoperacao'].agg(['min', 'max']))
    raise ValueError("Nenhum parque ativo no período")

# ==============================================================================
# 3. CALCULAR MÉDIA PONDERADA POR MÊS (PESO = CAPACIDADE INSTALADA)
# ==============================================================================
print("\n3. Calculando médias ponderadas por mês (apenas parques ativos)...")

# Variáveis meteorológicas para agregar
vars_meteo = [
    'vel_vento_10m',
    'vel_vento_100m',
    't2m',
    'sp',
    'tp',
    'sst',
    'zust',
    'densidade_ar',
    'wind_shear'
]

# Função para média ponderada
def weighted_average(df, var_col, weight_col='peso_capacidade_mw'):
    """Calcula média ponderada de uma variável"""
    # Filtrar valores válidos (não-NaN)
    valid = df[[var_col, weight_col]].dropna()
    if len(valid) == 0:
        return np.nan
    return (valid[var_col] * valid[weight_col]).sum() / valid[weight_col].sum()

# Agrupar por ano e mês
df_agregado = df_era5_ativos.groupby(['ano', 'mes']).apply(
    lambda x: pd.Series({
        **{f'{var}_ne_media': weighted_average(x, var) for var in vars_meteo},
        'capacidade_total_ne_mw': x['peso_capacidade_mw'].sum(),
        'num_parques_ativos': x['nome_parque'].nunique()
    })
).reset_index()

print(f"\n   Total de meses processados: {len(df_agregado)}")
print(f"\n   Primeiras 10 linhas:")
print(df_agregado.head(10).to_string())

# ==============================================================================
# 4. CALCULAR PERCENTIS DE VENTO (P10, P90) PARA CAPTURAR VARIABILIDADE
# ==============================================================================
print("\n4. Calculando percentis de vento...")

# Para cada mês, calcular P10 e P90 da velocidade do vento ponderados
def weighted_percentile(df, var_col, percentile, weight_col='peso_capacidade_mw'):
    """Calcula percentil ponderado"""
    valid = df[[var_col, weight_col]].dropna()
    if len(valid) == 0:
        return np.nan
    sorted_df = valid.sort_values(var_col)
    cumsum = sorted_df[weight_col].cumsum()
    cutoff = sorted_df[weight_col].sum() * (percentile / 100.0)
    idx = (cumsum >= cutoff).idxmax()
    return sorted_df.loc[idx, var_col]

df_percentis = df_era5_ativos.groupby(['ano', 'mes']).apply(
    lambda x: pd.Series({
        'vel_vento_100m_ne_p10': weighted_percentile(x, 'vel_vento_100m', 10),
        'vel_vento_100m_ne_p90': weighted_percentile(x, 'vel_vento_100m', 90),
        'vel_vento_10m_ne_p10': weighted_percentile(x, 'vel_vento_10m', 10),
        'vel_vento_10m_ne_p90': weighted_percentile(x, 'vel_vento_10m', 90),
    })
).reset_index()

# Merge com df_agregado
df_agregado = df_agregado.merge(df_percentis, on=['ano', 'mes'], how='left')

print(f"   Percentis calculados para {len(df_agregado)} meses")

# ==============================================================================
# 5. ADICIONAR FEATURES DERIVADAS
# ==============================================================================
print("\n5. Calculando features derivadas...")

# Potência do vento (0.5 * densidade * v^3)
df_agregado['potencia_vento_100m_ne'] = (
    0.5 * df_agregado['densidade_ar_ne_media'] * (df_agregado['vel_vento_100m_ne_media'] ** 3)
)

# Temperatura em Celsius (para facilitar interpretação)
df_agregado['temp_celsius_ne'] = df_agregado['t2m_ne_media'] - 273.15

# Precipitação acumulada mensal (tp já é média mensal em m)
df_agregado['precipitacao_mensal_mm'] = df_agregado['tp_ne_media'] * 1000  # converter m para mm

# Ratio de vento (100m vs 10m)
df_agregado['ratio_vento_100_10'] = (
    df_agregado['vel_vento_100m_ne_media'] / df_agregado['vel_vento_10m_ne_media'].replace(0, np.nan)
)

# Variabilidade do vento (P90 - P10)
df_agregado['variabilidade_vento_100m'] = (
    df_agregado['vel_vento_100m_ne_p90'] - df_agregado['vel_vento_100m_ne_p10']
)

print(f"   Features derivadas calculadas")

# ==============================================================================
# 6. ORDENAR E SELECIONAR COLUNAS FINAIS
# ==============================================================================
print("\n6. Preparando dataset final...")

colunas_finais = [
    'ano', 'mes',
    # Médias ponderadas
    'vel_vento_100m_ne_media',
    'vel_vento_10m_ne_media',
    't2m_ne_media',
    'temp_celsius_ne',
    'sp_ne_media',
    'sst_ne_media',
    'zust_ne_media',
    'densidade_ar_ne_media',
    'wind_shear_ne_media',
    # Percentis
    'vel_vento_100m_ne_p10',
    'vel_vento_100m_ne_p90',
    'vel_vento_10m_ne_p10',
    'vel_vento_10m_ne_p90',
    # Derivadas
    'potencia_vento_100m_ne',
    'precipitacao_mensal_mm',
    'ratio_vento_100_10',
    'variabilidade_vento_100m',
    # Metadados
    'capacidade_total_ne_mw',
    'num_parques_ativos'
]

df_agregado = df_agregado[colunas_finais]
df_agregado = df_agregado.sort_values(['ano', 'mes']).reset_index(drop=True)

print(f"\n   Dataset final: {df_agregado.shape}")
print(f"\n   Primeiras 10 linhas:")
print(df_agregado.head(10).to_string())
print(f"\n   Últimas 10 linhas:")
print(df_agregado.tail(10).to_string())

print(f"\n   Estatísticas:")
print(df_agregado.describe())

print(f"\n   Valores nulos por coluna:")
print(df_agregado.isnull().sum())

# ==============================================================================
# 7. SALVAR
# ==============================================================================
output_file = os.path.join(pasta_dados, 'dados_era5_ne_agregado.csv')
df_agregado.to_csv(output_file, index=False)

print(f"\n" + "="*100)
print(f"ARQUIVO SALVO: {output_file}")
print(f"Total de registros: {len(df_agregado)}")
print(f"Período: {df_agregado['ano'].min()}/{df_agregado['mes'].min():02d} a {df_agregado['ano'].max()}/{df_agregado['mes'].max():02d}")
print(f"\n✅ CORREÇÃO APLICADA:")
print(f"  - Filtro temporal: apenas parques ATIVOS em cada mês são incluídos")
print(f"  - Peso dinâmico: capacidade varia por mês conforme parques entram/saem")
print(f"  - Coluna 'num_parques_ativos' mostra quantos parques estavam ativos por mês")
print(f"\nColunas meteorológicas agregadas:")
for col in colunas_finais:
    if col not in ['ano', 'mes', 'capacidade_total_ne_mw', 'num_parques_ativos']:
        print(f"  - {col}")
print("="*100)



AGREGACAO ERA5 - MEDIA PONDERADA POR CAPACIDADE INSTALADA (DINAMICA)

1. Carregando dados ERA5 por parque...
   Total de registros: 319704
   Parques únicos: 2403
   Período: 2015/01 a 2025/12

2. Aplicando filtro temporal (parques ativos por mês)...
   Registros ANTES do filtro: 319704
   Registros DEPOIS do filtro (ativos): 66747
   Registros removidos (inativos): 252957
   % removido: 79.1%

3. Calculando médias ponderadas por mês (apenas parques ativos)...

   Total de meses processados: 132

   Primeiras 10 linhas:
    ano  mes  vel_vento_10m_ne_media  vel_vento_100m_ne_media  t2m_ne_media   sp_ne_media  tp_ne_media  sst_ne_media  zust_ne_media  densidade_ar_ne_media  wind_shear_ne_media  capacidade_total_ne_mw  num_parques_ativos
0  2015    1                5.948146                 7.639418    299.799070  99009.233073     0.000755    300.929169       0.301586               1.150351             0.121188               3409.4252               117.0
1  2015    2                5.0948

In [46]:
#Junção de datasets - Dados ONS + ERA5 por Parque (expandido em colunas)

import pandas as pd
import numpy as np
import os

print("="*100)
print("DATASET FINAL - ONS + DADOS METEOROLOGICOS POR PARQUE")
print("="*100)

# ==============================================================================
# 1. CARREGAR DATASETS
# ==============================================================================
print("1. Carregando datasets...")

# Dados ONS (geração, capacidade, demanda) - será nosso target
arquivo_ons = os.path.join(pasta_dados, 'dados_operacao_ONS.csv')
df_ons = pd.read_csv(arquivo_ons)
print(f"   Dados ONS: {df_ons.shape}")
print(f"   Período: {df_ons['ano'].min()}/{df_ons['mes'].min():02d} a {df_ons['ano'].max()}/{df_ons['mes'].max():02d}")

# Dados ERA5 por parque (NÃO agregados)
arquivo_era5 = os.path.join(pasta_dados, 'dados_era5_parques_ne.csv')
df_era5 = pd.read_csv(arquivo_era5)
print(f"   Dados ERA5 por parque: {df_era5.shape}")
print(f"   Parques únicos: {df_era5['nome_parque'].nunique()}")

# ==============================================================================
# 2. PREPARAR DADOS ERA5 PARA PIVOT (PARQUES -> COLUNAS)
# ==============================================================================
print("2. Preparando pivot dos dados ERA5...")

# Variáveis meteorológicas que queremos como features
vars_meteo = ['vel_vento_100m', 'vel_vento_10m', 't2m', 'sp', 'tp', 'sst', 'zust', 
              'densidade_ar', 'wind_shear']

# Criar identificador único para cada parque (usar cell_key que agrupa parques na mesma célula)
# Para não ter milhares de colunas, vamos usar cell_key (225 células únicas)
df_era5['cell_id'] = df_era5['cell_key']

print(f"   Células únicas (localizações): {df_era5['cell_id'].nunique()}")

# Para cada célula, pegar a média ponderada por capacidade dos parques naquela célula
# Isso mantém a granularidade espacial sem ter uma coluna por parque
df_era5_cell = df_era5.groupby(['ano', 'mes', 'cell_id']).apply(
    lambda x: pd.Series({
        **{var: np.average(x[var].dropna(), weights=x.loc[x[var].notna(), 'peso_capacidade_mw']) 
           if len(x[var].dropna()) > 0 and x.loc[x[var].notna(), 'peso_capacidade_mw'].sum() > 0 
           else np.nan 
           for var in vars_meteo},
        'capacidade_mw': x['peso_capacidade_mw'].sum()
    })
).reset_index()

print(f"   Dados agregados por célula: {df_era5_cell.shape}")

# ==============================================================================
# 3. PIVOT - CADA CÉLULA VIRA UM CONJUNTO DE COLUNAS
# ==============================================================================
print("3. Pivotando células para colunas...")

# Criar dataframe pivotado
dfs_pivot = []

for var in vars_meteo + ['capacidade_mw']:
    df_pivot = df_era5_cell.pivot(
        index=['ano', 'mes'], 
        columns='cell_id', 
        values=var
    )
    # Renomear colunas: var_cellkey
    df_pivot.columns = [f"{var}_{col}" for col in df_pivot.columns]
    dfs_pivot.append(df_pivot)

# Concatenar todos os pivots
df_era5_wide = pd.concat(dfs_pivot, axis=1).reset_index()

print(f"   ERA5 pivotado: {df_era5_wide.shape}")
print(f"   Total de colunas meteorológicas: {len(df_era5_wide.columns) - 2}")

# ==============================================================================
# 4. MERGE COM DADOS ONS
# ==============================================================================
print("4. Fazendo merge ONS + ERA5...")

df_final = df_ons.merge(df_era5_wide, on=['ano', 'mes'], how='inner')

print(f"   Dataset após merge: {df_final.shape}")
print(f"   Linhas (meses): {len(df_final)}")
print(f"   Colunas totais: {len(df_final.columns)}")

# ==============================================================================
# 5. CRIAR COLUNA DE DATA
# ==============================================================================
print("5. Criando coluna de data...")

df_final['data'] = pd.to_datetime(
    df_final['ano'].astype(str) + '-' + df_final['mes'].astype(str).str.zfill(2) + '-01'
)

# Reordenar colunas
cols_ons = [c for c in df_ons.columns if c in df_final.columns]
cols_meteo = [c for c in df_final.columns if c not in cols_ons and c != 'data']
df_final = df_final[['data'] + cols_ons + cols_meteo]

# ==============================================================================
# 6. ESTATÍSTICAS
# ==============================================================================
print("6. Estatísticas do dataset final...")

print(f"Shape final: {df_final.shape}")
print(f"   Período: {df_final['data'].min()} a {df_final['data'].max()}")

# Contar colunas por categoria
n_ons = len(cols_ons)
n_meteo = len([c for c in df_final.columns if any(v in c for v in vars_meteo)])
n_cap = len([c for c in df_final.columns if 'capacidade_mw_' in c])

print(f"Colunas por categoria:")
print(f"     - Data: 1")
print(f"     - ONS (operação): {n_ons}")
print(f"     - Meteorológicas: {n_meteo}")
print(f"     - Capacidade por célula: {n_cap}")

print(f"Target para modelagem: 'geracao_eolica_ne_mwmed'")

# Valores nulos
nulos = df_final.isnull().sum()
cols_com_nulos = nulos[nulos > 0]
print(f"Colunas com valores nulos: {len(cols_com_nulos)}")
if len(cols_com_nulos) > 0:
    print(f"   (SST tem nulos em células continentais - normal)")

# ==============================================================================
# 7. SALVAR DATASET FINAL
# ==============================================================================
print("7. Salvando dataset final...")

output_csv = os.path.join(pasta_dados, 'dataset_final_modelagem.csv')
df_final.to_csv(output_csv, index=False)
print(f"   CSV: {output_csv}")

output_parquet = os.path.join(pasta_dados, 'dataset_final_modelagem.parquet')
df_final.to_parquet(output_parquet, index=False)
print(f"   Parquet: {output_parquet}")

# ==============================================================================
# RESUMO
# ==============================================================================
print("" + "="*100)
print("DATASET FINAL CRIADO!")
print("="*100)
print(f"Formato do dataset:")
print(f"  - Linhas: {len(df_final)} meses (2015-01 a 2025-12)")
print(f"  - Colunas: {len(df_final.columns)} (ONS + meteorológicas por célula)")
print(f"Target: geracao_eolica_ne_mwmed")
print(f"Features: dados meteorológicos de {df_era5['cell_id'].nunique()} células/localizações")
print("="*100)



DATASET FINAL - ONS + DADOS METEOROLOGICOS POR PARQUE
1. Carregando datasets...
   Dados ONS: (132, 14)
   Período: 2015/01 a 2025/12
   Dados ERA5 por parque: (319704, 26)
   Parques únicos: 2403
2. Preparando pivot dos dados ERA5...
   Células únicas (localizações): 225
   Dados agregados por célula: (29700, 13)
3. Pivotando células para colunas...
   ERA5 pivotado: (132, 2252)
   Total de colunas meteorológicas: 2250
4. Fazendo merge ONS + ERA5...
   Dataset após merge: (132, 2264)
   Linhas (meses): 132
   Colunas totais: 2264
5. Criando coluna de data...
6. Estatísticas do dataset final...
Shape final: (132, 2265)
   Período: 2015-01-01 00:00:00 a 2025-12-01 00:00:00
Colunas por categoria:
     - Data: 1
     - ONS (operação): 14
     - Meteorológicas: 2025
     - Capacidade por célula: 225
Target para modelagem: 'geracao_eolica_ne_mwmed'
Colunas com valores nulos: 552
   (SST tem nulos em células continentais - normal)
7. Salvando dataset final...
   CSV: C:\\Users\\Admin\\Docume

In [47]:
## Preparação do INPUT do modelo (algoritmos de padronização minMax...)

In [48]:
#EDA COMPLETA - DATASET FINAL DE MODELAGEM


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.signal import find_peaks
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import warnings
warnings.filterwarnings('ignore')
import os
from datetime import datetime

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================

# Caminhos
DADOS_DIR = r'C:\Users\Admin\Documents\Puc\IC\Dados'
DATASET_FILE = os.path.join(DADOS_DIR, 'dataset_final_modelagem.parquet')
EDA_DIR = os.path.join(DADOS_DIR, 'EDA_Completa')
os.makedirs(EDA_DIR, exist_ok=True)

# Criar subdiretórios
subdirs = [
    'validacao',
    'series_temporais',
    'estatisticas',
    'sazonalidade',
    'mapas_espaciais',
    'heatmaps',
    'decomposicao',
    'correlacoes',
    'extremos',
    'indicadores',
    'hipoteses'
]
for subdir in subdirs:
    os.makedirs(os.path.join(EDA_DIR, subdir), exist_ok=True)

print("="*100)
print("EDA COMPLETA - DATASET FINAL DE MODELAGEM")
print("="*100)

# ==============================================================================
# 1. CARREGAMENTO E VALIDAÇÃO DO DICIONÁRIO DE DADOS
# ==============================================================================
print("\n1. CARREGAMENTO E VALIDAÇÃO DO DICIONÁRIO DE DADOS")
print("-"*100)

# Carregar dataset
df = pd.read_parquet(DATASET_FILE)
print(f"✓ Dataset carregado: {df.shape[0]} linhas × {df.shape[1]} colunas")

# Converter data para datetime se necessário
if 'data' not in df.columns and 'ano' in df.columns and 'mes' in df.columns:
    df['data'] = pd.to_datetime(df['ano'].astype(str) + '-' + df['mes'].astype(str).str.zfill(2) + '-01')

df = df.sort_values('data').reset_index(drop=True)

# Dicionário de dados
dicionario_dados = {
    # Temporais
    'data': {'unidade': 'datetime', 'descricao': 'Data de referência (YYYY-MM-01)', 'tipo': 'temporal'},
    'ano': {'unidade': 'ano', 'descricao': 'Ano', 'tipo': 'temporal'},
    'mes': {'unidade': 'mês', 'descricao': 'Mês (1-12)', 'tipo': 'temporal'},

    # ONS - Geração e Capacidade
    'geracao_eolica_ne_mwmed': {'unidade': 'MWmed', 'descricao': 'Geração eólica média mensal NE', 'tipo': 'target'},
    'capacidade_eolica_ne_mw': {'unidade': 'MW', 'descricao': 'Capacidade instalada acumulada NE', 'tipo': 'ons'},
    'crescimento_capacidade_mw': {'unidade': 'MW', 'descricao': 'Capacidade adicionada no mês', 'tipo': 'ons'},

    # ONS - Demanda
    'carga_ne_mwmed': {'unidade': 'MWmed', 'descricao': 'Carga média mensal do NE', 'tipo': 'ons'},
    'demanda_sin_mwmed': {'unidade': 'MWmed', 'descricao': 'Demanda média mensal do SIN', 'tipo': 'ons'},
    'demanda_max_sin_mw': {'unidade': 'MW', 'descricao': 'Pico de demanda mensal do SIN', 'tipo': 'ons'},

    # ONS - Indicadores
    'corte_eolica_ne_mwmed': {'unidade': 'MWmed', 'descricao': 'Curtailment estimado (CF=45%)', 'tipo': 'ons'},
    'capacity_factor_ne': {'unidade': '%', 'descricao': 'Fator de capacidade mensal NE', 'tipo': 'indicador'},
    'penetracao_eolica_ne': {'unidade': '%', 'descricao': 'Penetração eólica no NE', 'tipo': 'indicador'},
    'penetracao_eolica_sin': {'unidade': '%', 'descricao': 'Penetração eólica no SIN', 'tipo': 'indicador'},

    # ERA5 - Vento
    'vel_vento_100m_ne_media': {'unidade': 'm/s', 'descricao': 'Velocidade média do vento a 100m', 'tipo': 'era5'},
    'vel_vento_10m_ne_media': {'unidade': 'm/s', 'descricao': 'Velocidade média do vento a 10m', 'tipo': 'era5'},
    'vel_vento_100m_ne_p10': {'unidade': 'm/s', 'descricao': 'Percentil 10 do vento a 100m', 'tipo': 'era5'},
    'vel_vento_100m_ne_p90': {'unidade': 'm/s', 'descricao': 'Percentil 90 do vento a 100m', 'tipo': 'era5'},
    'variabilidade_vento_100m': {'unidade': 'm/s', 'descricao': 'Variabilidade do vento (P90-P10)', 'tipo': 'era5'},

    # ERA5 - Atmosféricos
    't2m_ne_media': {'unidade': 'K', 'descricao': 'Temperatura média a 2m', 'tipo': 'era5'},
    'temp_celsius_ne': {'unidade': '°C', 'descricao': 'Temperatura média em Celsius', 'tipo': 'era5'},
    'sp_ne_media': {'unidade': 'Pa', 'descricao': 'Pressão superficial média', 'tipo': 'era5'},
    'densidade_ar_ne_media': {'unidade': 'kg/m³', 'descricao': 'Densidade do ar média', 'tipo': 'era5'},
    'precipitacao_mensal_mm': {'unidade': 'mm', 'descricao': 'Precipitação acumulada mensal', 'tipo': 'era5'},

    # ERA5 - Derivadas
    'potencia_vento_100m_ne': {'unidade': 'W/m²', 'descricao': 'Potência do vento (0.5*ρ*v³)', 'tipo': 'derivada'},
    'wind_shear_ne_media': {'unidade': 'adimensional', 'descricao': 'Wind shear médio', 'tipo': 'era5'},
    'ratio_vento_100_10': {'unidade': 'adimensional', 'descricao': 'Razão vento 100m/10m', 'tipo': 'derivada'},
}

# Relatório de validação
with open(os.path.join(EDA_DIR, 'validacao', '00_dicionario_dados.txt'), 'w', encoding='utf-8') as f:
    f.write("="*100 + "\n")
    f.write("DICIONÁRIO DE DADOS - DATASET FINAL DE MODELAGEM\n")
    f.write("="*100 + "\n\n")

    f.write(f"Dataset: {DATASET_FILE}\n")
    f.write(f"Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas\n")
    f.write(f"Período: {df['data'].min().strftime('%Y-%m')} a {df['data'].max().strftime('%Y-%m')}\n")
    f.write(f"Meses totais: {len(df)}\n\n")

    f.write("-"*100 + "\n")
    f.write("VARIÁVEIS DO DATASET\n")
    f.write("-"*100 + "\n\n")

    for col in df.columns:
        if col in dicionario_dados:
            info = dicionario_dados[col]
            f.write(f"• {col}\n")
            f.write(f"  Tipo: {info['tipo']}\n")
            f.write(f"  Unidade: {info['unidade']}\n")
            f.write(f"  Descrição: {info['descricao']}\n")
            f.write(f"  Dtype: {df[col].dtype}\n")
            f.write(f"  Nulos: {df[col].isnull().sum()} ({df[col].isnull().sum()/len(df)*100:.2f}%)\n\n")
        else:
            f.write(f"• {col}\n")
            f.write(f"  [Sem descrição no dicionário]\n")
            f.write(f"  Dtype: {df[col].dtype}\n")
            f.write(f"  Nulos: {df[col].isnull().sum()} ({df[col].isnull().sum()/len(df)*100:.2f}%)\n\n")

print(f"✓ Dicionário de dados salvo em: validacao/00_dicionario_dados.txt")

# ==============================================================================
# 2. CONSISTÊNCIA E INTEGRIDADE
# ==============================================================================
print("\n2. CONSISTÊNCIA E INTEGRIDADE")
print("-"*100)

# Verificar sequência temporal
datas_esperadas = pd.date_range(start=df['data'].min(), end=df['data'].max(), freq='MS')
datas_faltantes = set(datas_esperadas) - set(df['data'])

print(f"✓ Sequência temporal: {len(datas_faltantes)} meses faltantes")

# Verificar duplicatas
duplicatas = df.duplicated(subset=['ano', 'mes']).sum()
print(f"✓ Duplicatas: {duplicatas} registros duplicados")

# Estatísticas de missing
missing_por_coluna = df.isnull().sum()
missing_total = missing_por_coluna.sum()
print(f"✓ Valores faltantes: {missing_total} células ({missing_total/(len(df)*len(df.columns))*100:.2f}%)")

# Detectar outliers (método IQR)
def detectar_outliers_iqr(serie):
    Q1 = serie.quantile(0.25)
    Q3 = serie.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return ((serie < lower) | (serie > upper)).sum()

# Detectar outliers (método Z-score)
def detectar_outliers_zscore(serie, threshold=3):
    z_scores = np.abs(stats.zscore(serie.dropna()))
    return (z_scores > threshold).sum()

# Relatório de outliers
colunas_numericas = df.select_dtypes(include=[np.number]).columns
colunas_para_outliers = [col for col in colunas_numericas if col not in ['ano', 'mes']]

outliers_report = []
for col in colunas_para_outliers[:20]:  # Limitar para não sobrecarregar
    outliers_iqr = detectar_outliers_iqr(df[col])
    outliers_z = detectar_outliers_zscore(df[col])
    outliers_report.append({
        'Variável': col,
        'Outliers (IQR)': outliers_iqr,
        'Outliers (Z-score>3)': outliers_z
    })

df_outliers = pd.DataFrame(outliers_report)
df_outliers.to_csv(os.path.join(EDA_DIR, 'validacao', '01_outliers_report.csv'), index=False)
print(f"✓ Relatório de outliers salvo em: validacao/01_outliers_report.csv")

# Salvar relatório de integridade
with open(os.path.join(EDA_DIR, 'validacao', '02_integridade.txt'), 'w', encoding='utf-8') as f:
    f.write("="*100 + "\n")
    f.write("RELATÓRIO DE INTEGRIDADE E CONSISTÊNCIA\n")
    f.write("="*100 + "\n\n")

    f.write("1. COBERTURA TEMPORAL\n")
    f.write(f"   Período: {df['data'].min().strftime('%Y-%m')} a {df['data'].max().strftime('%Y-%m')}\n")
    f.write(f"   Meses esperados: {len(datas_esperadas)}\n")
    f.write(f"   Meses presentes: {len(df)}\n")
    f.write(f"   Meses faltantes: {len(datas_faltantes)}\n")
    if datas_faltantes:
        f.write(f"   Datas faltantes: {sorted([d.strftime('%Y-%m') for d in datas_faltantes])}\n")
    f.write("\n")

    f.write("2. DUPLICATAS\n")
    f.write(f"   Registros duplicados: {duplicatas}\n\n")

    f.write("3. VALORES FALTANTES\n")
    for col, count in missing_por_coluna[missing_por_coluna > 0].items():
        f.write(f"   {col}: {count} ({count/len(df)*100:.2f}%)\n")
    f.write("\n")

    f.write("4. OUTLIERS (TOP 10 variáveis com mais outliers)\n")
    top_outliers = df_outliers.sort_values('Outliers (IQR)', ascending=False).head(10)
    f.write(top_outliers.to_string(index=False))
    f.write("\n")

print(f"✓ Relatório de integridade salvo em: validacao/02_integridade.txt")

# ==============================================================================
# 3. SÉRIES TEMPORAIS PRINCIPAIS (Estilo ONS)
# ==============================================================================
print("\n3. SÉRIES TEMPORAIS PRINCIPAIS")
print("-"*100)

# Variáveis principais para plotar
variaveis_principais = {
    'capacidade_eolica_ne_mw': 'Capacidade Eólica Instalada NE (MW)',
    'geracao_eolica_ne_mwmed': 'Geração Eólica Média Mensal NE (MWmed)',
    'carga_ne_mwmed': 'Carga Média Mensal NE (MWmed)',
    'demanda_sin_mwmed': 'Demanda Média Mensal SIN (MWmed)',
    'demanda_max_sin_mw': 'Demanda Máxima Mensal SIN (MW)',
    'corte_eolica_ne_mwmed': 'Curtailment Estimado NE (MWmed)',
    'capacity_factor_ne': 'Fator de Capacidade NE (%)',
    'vel_vento_100m_ne_media': 'Velocidade do Vento a 100m NE (m/s)'
}

for i, (col, titulo) in enumerate(variaveis_principais.items(), 1):
    if col not in df.columns:
        continue

    fig, ax = plt.subplots(figsize=(16, 6))

    ax.plot(df['data'], df[col], linewidth=2, marker='o', markersize=4, label=titulo)

    # Adicionar linha de tendência
    z = np.polyfit(range(len(df)), df[col].fillna(df[col].mean()), 1)
    p = np.poly1d(z)
    ax.plot(df['data'], p(range(len(df))),
            linestyle='--', linewidth=2, color='red', alpha=0.7, label='Tendência Linear')

    ax.set_xlabel('Data', fontsize=12, fontweight='bold')
    ax.set_ylabel(titulo.split('(')[1].replace(')', ''), fontsize=12, fontweight='bold')
    ax.set_title(f'{titulo}\nPeríodo: {df["data"].min().strftime("%b/%Y")} a {df["data"].max().strftime("%b/%Y")}',
                 fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=11)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(EDA_DIR, 'series_temporais', f'{i:02d}_{col}.png'), dpi=300, bbox_inches='tight')
    plt.close()

print(f"✓ {len(variaveis_principais)} séries temporais salvas em: series_temporais/")

# ==============================================================================
# 4. ESTATÍSTICAS DESCRITIVAS COMPLETAS
# ==============================================================================
print("\n4. ESTATÍSTICAS DESCRITIVAS COMPLETAS")
print("-"*100)

def estatisticas_completas(serie):
    """Calcula estatísticas descritivas completas para uma série"""
    serie_clean = serie.dropna()

    return {
        'count': len(serie_clean),
        'mean': serie_clean.mean(),
        'median': serie_clean.median(),
        'min': serie_clean.min(),
        'max': serie_clean.max(),
        'std': serie_clean.std(),
        'var': serie_clean.var(),
        'q25': serie_clean.quantile(0.25),
        'q75': serie_clean.quantile(0.75),
        'iqr': serie_clean.quantile(0.75) - serie_clean.quantile(0.25),
        'p10': serie_clean.quantile(0.10),
        'p50': serie_clean.quantile(0.50),
        'p90': serie_clean.quantile(0.90),
        'cv': serie_clean.std() / serie_clean.mean() if serie_clean.mean() != 0 else np.nan,
        'skewness': stats.skew(serie_clean),
        'kurtosis': stats.kurtosis(serie_clean),
        'range': serie_clean.max() - serie_clean.min(),
    }

# Calcular estatísticas para variáveis principais
stats_results = []
for col in colunas_para_outliers[:30]:  # Top 30 variáveis
    if col in df.columns:
        stats_dict = estatisticas_completas(df[col])
        stats_dict['variavel'] = col
        stats_results.append(stats_dict)

df_stats = pd.DataFrame(stats_results)
df_stats = df_stats[['variavel', 'count', 'mean', 'median', 'std', 'min', 'max',
                      'q25', 'q75', 'iqr', 'p10', 'p50', 'p90',
                      'cv', 'skewness', 'kurtosis', 'range']]

df_stats.to_csv(os.path.join(EDA_DIR, 'estatisticas', '01_estatisticas_descritivas.csv'), index=False)
print(f"✓ Estatísticas descritivas salvas em: estatisticas/01_estatisticas_descritivas.csv")

# Criar visualização das distribuições
fig, axes = plt.subplots(4, 2, figsize=(16, 16))
axes = axes.ravel()

variaveis_distribuicao = [
    'geracao_eolica_ne_mwmed',
    'capacity_factor_ne',
    'vel_vento_100m_ne_media',
    'carga_ne_mwmed',
    'penetracao_eolica_ne',
    'corte_eolica_ne_mwmed',
    'temp_celsius_ne',
    'densidade_ar_ne_media'
]

for i, col in enumerate(variaveis_distribuicao):
    if col in df.columns and i < len(axes):
        ax = axes[i]
        data = df[col].dropna()

        # Histograma + KDE
        ax.hist(data, bins=30, density=True, alpha=0.6, color='skyblue', edgecolor='black')

        # KDE
        try:
            from scipy.stats import gaussian_kde
            kde = gaussian_kde(data)
            x_range = np.linspace(data.min(), data.max(), 100)
            ax.plot(x_range, kde(x_range), 'r-', linewidth=2, label='KDE')
        except:
            pass

        ax.set_xlabel(col, fontsize=10)
        ax.set_ylabel('Densidade', fontsize=10)
        ax.set_title(f'Distribuição: {col}', fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3)

        # Adicionar estatísticas no gráfico
        stats_text = f"μ={data.mean():.2f}\nσ={data.std():.2f}\nSkew={stats.skew(data):.2f}"
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes,
                verticalalignment='top', fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'estatisticas', '02_distribuicoes.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Gráfico de distribuições salvo em: estatisticas/02_distribuicoes.png")

# ==============================================================================
# 5. TENDÊNCIAS E TAXAS DE CRESCIMENTO
# ==============================================================================
print("\n5. TENDÊNCIAS E TAXAS DE CRESCIMENTO")
print("-"*100)

def calcular_tendencia(serie, datas):
    """Calcula tendência linear e taxa de crescimento"""
    serie_clean = serie.dropna()
    x = np.arange(len(serie_clean))

    # Regressão linear
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, serie_clean)

    # CAGR (Compound Annual Growth Rate)
    if serie_clean.iloc[0] > 0 and serie_clean.iloc[-1] > 0:
        n_years = len(serie_clean) / 12
        cagr = (pow(serie_clean.iloc[-1] / serie_clean.iloc[0], 1/n_years) - 1) * 100
    else:
        cagr = np.nan

    return {
        'slope': slope,
        'intercept': intercept,
        'r_squared': r_value**2,
        'p_value': p_value,
        'cagr_%': cagr,
        'crescimento_total_%': ((serie_clean.iloc[-1] - serie_clean.iloc[0]) / serie_clean.iloc[0] * 100) if serie_clean.iloc[0] > 0 else np.nan
    }

# Calcular tendências
tendencias = []
for col in ['capacidade_eolica_ne_mw', 'geracao_eolica_ne_mwmed', 'carga_ne_mwmed',
            'demanda_sin_mwmed', 'vel_vento_100m_ne_media', 'capacity_factor_ne']:
    if col in df.columns:
        tend = calcular_tendencia(df[col], df['data'])
        tend['variavel'] = col
        tendencias.append(tend)

df_tendencias = pd.DataFrame(tendencias)
df_tendencias = df_tendencias[['variavel', 'slope', 'intercept', 'r_squared', 'p_value', 'cagr_%', 'crescimento_total_%']]
df_tendencias.to_csv(os.path.join(EDA_DIR, 'estatisticas', '03_tendencias.csv'), index=False)

print(f"✓ Análise de tendências salva em: estatisticas/03_tendencias.csv")

# Taxas de crescimento YoY (Year-over-Year)
df['ano_mes'] = df['data'].dt.to_period('M')
df['capacidade_yoy'] = df['capacidade_eolica_ne_mw'].pct_change(12) * 100
df['geracao_yoy'] = df['geracao_eolica_ne_mwmed'].pct_change(12) * 100

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Capacidade YoY
axes[0].plot(df['data'], df['capacidade_yoy'], marker='o', linewidth=2, color='navy')
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[0].set_ylabel('Crescimento YoY (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Taxa de Crescimento Year-over-Year - Capacidade Eólica NE', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Geração YoY
axes[1].plot(df['data'], df['geracao_yoy'], marker='o', linewidth=2, color='darkgreen')
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[1].set_xlabel('Data', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Crescimento YoY (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Taxa de Crescimento Year-over-Year - Geração Eólica NE', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'estatisticas', '04_crescimento_yoy.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Gráfico de crescimento YoY salvo em: estatisticas/04_crescimento_yoy.png")

# ==============================================================================
# 6. SAZONALIDADE POR MÊS DO ANO
# ==============================================================================
print("\n6. SAZONALIDADE POR MÊS DO ANO")
print("-"*100)

# Criar boxplots por mês
fig, axes = plt.subplots(3, 2, figsize=(18, 14))
axes = axes.ravel()

variaveis_sazonalidade = [
    'geracao_eolica_ne_mwmed',
    'capacity_factor_ne',
    'vel_vento_100m_ne_media',
    'carga_ne_mwmed',
    'penetracao_eolica_ne',
    'temp_celsius_ne'
]

for i, col in enumerate(variaveis_sazonalidade):
    if col in df.columns and i < len(axes):
        ax = axes[i]

        # Boxplot por mês
        df.boxplot(column=col, by='mes', ax=ax, grid=True)
        ax.set_xlabel('Mês', fontsize=11, fontweight='bold')
        ax.set_ylabel(col, fontsize=11)
        ax.set_title(f'Sazonalidade Mensal: {col}', fontsize=12, fontweight='bold')
        ax.get_figure().suptitle('')  # Remove título automático do pandas

        # Adicionar linha de média mensal
        medias_mensais = df.groupby('mes')[col].mean()
        ax.plot(range(1, 13), medias_mensais, 'r-o', linewidth=2, markersize=8, label='Média')
        ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'sazonalidade', '01_boxplots_mensais.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Boxplots mensais salvos em: sazonalidade/01_boxplots_mensais.png")

# Perfis sazonais (médias mensais)
perfis_sazonais = df.groupby('mes')[variaveis_sazonalidade].mean()
perfis_sazonais.to_csv(os.path.join(EDA_DIR, 'sazonalidade', '02_perfis_sazonais.csv'))

# Gráfico de perfis sazonais
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

meses_nomes = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

for i, col in enumerate(variaveis_sazonalidade):
    if i < len(axes):
        ax = axes[i]
        perfil = perfis_sazonais[col]

        ax.plot(range(1, 13), perfil, marker='o', linewidth=3, markersize=10, color='darkblue')
        ax.fill_between(range(1, 13), perfil, alpha=0.3)
        ax.set_xlabel('Mês', fontsize=11, fontweight='bold')
        ax.set_ylabel('Média', fontsize=11, fontweight='bold')
        ax.set_title(f'{col}', fontsize=12, fontweight='bold')
        ax.set_xticks(range(1, 13))
        ax.set_xticklabels(meses_nomes, rotation=45)
        ax.grid(True, alpha=0.3)

        # Destacar máximo e mínimo
        idx_max = perfil.idxmax()
        idx_min = perfil.idxmin()
        ax.scatter([idx_max], [perfil.loc[idx_max]], color='red', s=200, zorder=5, label=f'Max: {meses_nomes[idx_max-1]}')
        ax.scatter([idx_min], [perfil.loc[idx_min]], color='green', s=200, zorder=5, label=f'Min: {meses_nomes[idx_min-1]}')
        ax.legend(loc='best', fontsize=9)

plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'sazonalidade', '03_perfis_sazonais.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Perfis sazonais salvos em: sazonalidade/")

# ==============================================================================
# 7. HEATMAPS ANO × MÊS
# ==============================================================================
print("\n7. HEATMAPS ANO × MÊS")
print("-"*100)

variaveis_heatmap = [
    'geracao_eolica_ne_mwmed',
    'vel_vento_100m_ne_media',
    'carga_ne_mwmed',
    'capacity_factor_ne',
    'penetracao_eolica_ne',
    'corte_eolica_ne_mwmed'
]

for col in variaveis_heatmap:
    if col not in df.columns:
        continue

    # Criar pivot table
    pivot = df.pivot_table(values=col, index='ano', columns='mes', aggfunc='mean')

    fig, ax = plt.subplots(figsize=(14, 8))

    sns.heatmap(pivot, annot=True, fmt='.1f', cmap='YlOrRd', linewidths=0.5,
                cbar_kws={'label': col}, ax=ax)

    ax.set_xlabel('Mês', fontsize=12, fontweight='bold')
    ax.set_ylabel('Ano', fontsize=12, fontweight='bold')
    ax.set_title(f'Heatmap Ano × Mês: {col}', fontsize=14, fontweight='bold')
    ax.set_xticklabels(meses_nomes, rotation=0)

    plt.tight_layout()
    plt.savefig(os.path.join(EDA_DIR, 'heatmaps', f'{col}_heatmap.png'), dpi=300, bbox_inches='tight')
    plt.close()

print(f"✓ {len(variaveis_heatmap)} heatmaps salvos em: heatmaps/")

# ==============================================================================
# 8. DECOMPOSIÇÃO DE SÉRIES TEMPORAIS
# ==============================================================================
print("\n8. DECOMPOSIÇÃO DE SÉRIES TEMPORAIS")
print("-"*100)

variaveis_decomposicao = [
    'geracao_eolica_ne_mwmed',
    'vel_vento_100m_ne_media',
    'capacity_factor_ne',
    'carga_ne_mwmed'
]

for col in variaveis_decomposicao:
    if col not in df.columns:
        continue

    try:
        # Preparar série
        serie = df.set_index('data')[col].dropna()

        if len(serie) < 24:  # Precisa de pelo menos 2 anos
            continue

        # Decomposição STL (Seasonal-Trend decomposition using Loess)
        from statsmodels.tsa.seasonal import STL
        stl = STL(serie, seasonal=13)  # seasonal = 13 para capturar ciclo anual
        result = stl.fit()

        # Plotar
        fig = result.plot()
        fig.set_size_inches(16, 10)
        fig.suptitle(f'Decomposição STL: {col}', fontsize=16, fontweight='bold', y=0.995)

        plt.tight_layout()
        plt.savefig(os.path.join(EDA_DIR, 'decomposicao', f'{col}_stl.png'), dpi=300, bbox_inches='tight')
        plt.close()

        # Salvar componentes
        componentes = pd.DataFrame({
            'data': serie.index,
            'observado': serie.values,
            'tendencia': result.trend,
            'sazonalidade': result.seasonal,
            'residuo': result.resid
        })
        componentes.to_csv(os.path.join(EDA_DIR, 'decomposicao', f'{col}_componentes.csv'), index=False)

    except Exception as e:
        print(f"   Aviso: Não foi possível decompor {col}: {e}")
        continue

print(f"✓ Decomposições STL salvas em: decomposicao/")

# ==============================================================================
# 9. ACF/PACF E CORRELAÇÕES CRUZADAS
# ==============================================================================
print("\n9. ACF/PACF E CORRELAÇÕES CRUZADAS")
print("-"*100)

variaveis_acf = [
    'geracao_eolica_ne_mwmed',
    'vel_vento_100m_ne_media',
    'capacity_factor_ne'
]

for col in variaveis_acf:
    if col not in df.columns:
        continue

    serie = df[col].dropna()

    fig, axes = plt.subplots(2, 1, figsize=(14, 8))

    # ACF
    plot_acf(serie, lags=40, ax=axes[0])
    axes[0].set_title(f'Autocorrelação (ACF): {col}', fontsize=14, fontweight='bold')

    # PACF
    plot_pacf(serie, lags=40, ax=axes[1])
    axes[1].set_title(f'Autocorrelação Parcial (PACF): {col}', fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.savefig(os.path.join(EDA_DIR, 'correlacoes', f'{col}_acf_pacf.png'), dpi=300, bbox_inches='tight')
    plt.close()

print(f"✓ Gráficos ACF/PACF salvos em: correlacoes/")

# Correlação cruzada entre geração e vento com lags
if 'geracao_eolica_ne_mwmed' in df.columns and 'vel_vento_100m_ne_media' in df.columns:
    serie1 = df['geracao_eolica_ne_mwmed'].dropna()
    serie2 = df['vel_vento_100m_ne_media'].dropna()

    # Alinhar séries
    min_len = min(len(serie1), len(serie2))
    serie1 = serie1[:min_len]
    serie2 = serie2[:min_len]

    # Calcular cross-correlation
    cross_corr = [np.corrcoef(serie1[lag:], serie2[:len(serie2)-lag])[0, 1]
                  for lag in range(0, 13)]

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(range(13), cross_corr, color='steelblue', edgecolor='black')
    ax.set_xlabel('Lag (meses)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Correlação', fontsize=12, fontweight='bold')
    ax.set_title('Correlação Cruzada: Geração Eólica × Velocidade do Vento', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    ax.axhline(y=0, color='red', linestyle='--', linewidth=1)

    # Adicionar valores
    for i, v in enumerate(cross_corr):
        ax.text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10)

    plt.tight_layout()
    plt.savefig(os.path.join(EDA_DIR, 'correlacoes', 'cross_correlation_geracao_vento.png'), dpi=300, bbox_inches='tight')
    plt.close()

    print(f"✓ Correlação cruzada salva em: correlacoes/cross_correlation_geracao_vento.png")

# ==============================================================================
# 10. ANÁLISE DE EXTREMOS
# ==============================================================================
print("\n10. ANÁLISE DE EXTREMOS")
print("-"*100)

extremos_report = []

for col in ['geracao_eolica_ne_mwmed', 'vel_vento_100m_ne_media',
            'capacity_factor_ne', 'carga_ne_mwmed']:
    if col not in df.columns:
        continue

    # Encontrar máximos e mínimos
    idx_max = df[col].idxmax()
    idx_min = df[col].idxmin()

    extremos_report.append({
        'variavel': col,
        'valor_max': df.loc[idx_max, col],
        'data_max': df.loc[idx_max, 'data'].strftime('%Y-%m'),
        'valor_min': df.loc[idx_min, col],
        'data_min': df.loc[idx_min, 'data'].strftime('%Y-%m'),
    })

df_extremos = pd.DataFrame(extremos_report)
df_extremos.to_csv(os.path.join(EDA_DIR, 'extremos', '01_recordes_mensais.csv'), index=False)

print(f"✓ Análise de extremos salva em: extremos/01_recordes_mensais.csv")

# Distribuição de extremos ao longo do tempo
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.ravel()

variaveis_extremos = ['geracao_eolica_ne_mwmed', 'vel_vento_100m_ne_media',
                      'capacity_factor_ne', 'carga_ne_mwmed']

for i, col in enumerate(variaveis_extremos):
    if col in df.columns and i < len(axes):
        ax = axes[i]

        # Plotar série
        ax.plot(df['data'], df[col], linewidth=1.5, color='gray', alpha=0.7)

        # Destacar top 5% e bottom 5%
        threshold_high = df[col].quantile(0.95)
        threshold_low = df[col].quantile(0.05)

        extremos_high = df[df[col] >= threshold_high]
        extremos_low = df[df[col] <= threshold_low]

        ax.scatter(extremos_high['data'], extremos_high[col],
                  color='red', s=80, zorder=5, label='Top 5%', alpha=0.8)
        ax.scatter(extremos_low['data'], extremos_low[col],
                  color='blue', s=80, zorder=5, label='Bottom 5%', alpha=0.8)

        ax.axhline(y=threshold_high, color='red', linestyle='--', linewidth=1, alpha=0.5)
        ax.axhline(y=threshold_low, color='blue', linestyle='--', linewidth=1, alpha=0.5)

        ax.set_xlabel('Data', fontsize=11, fontweight='bold')
        ax.set_ylabel(col, fontsize=11)
        ax.set_title(f'Eventos Extremos: {col}', fontsize=12, fontweight='bold')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'extremos', '02_eventos_extremos.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Gráfico de eventos extremos salvo em: extremos/02_eventos_extremos.png")

# ==============================================================================
# 11. MATRIZ DE CORRELAÇÃO E SCATTER PLOTS
# ==============================================================================
print("\n11. MATRIZ DE CORRELAÇÃO E SCATTER PLOTS")
print("-"*100)

# Selecionar variáveis principais para correlação
variaveis_correlacao = [
    'geracao_eolica_ne_mwmed',
    'capacidade_eolica_ne_mw',
    'vel_vento_100m_ne_media',
    'capacity_factor_ne',
    'carga_ne_mwmed',
    'demanda_sin_mwmed',
    'penetracao_eolica_ne',
    'corte_eolica_ne_mwmed',
    'temp_celsius_ne',
    'densidade_ar_ne_media',
    'potencia_vento_100m_ne',
    'variabilidade_vento_100m'
]

# Filtrar variáveis existentes
variaveis_correlacao = [v for v in variaveis_correlacao if v in df.columns]

# Matriz de correlação
df_corr = df[variaveis_correlacao].corr()

# Plotar matriz de correlação
fig, ax = plt.subplots(figsize=(14, 12))
mask = np.triu(np.ones_like(df_corr, dtype=bool))
sns.heatmap(df_corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, linewidths=1, cbar_kws={'label': 'Correlação'}, ax=ax,
            vmin=-1, vmax=1)
ax.set_title('Matriz de Correlação - Variáveis Principais', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'correlacoes', '01_matriz_correlacao.png'), dpi=300, bbox_inches='tight')
plt.close()

# Salvar matriz em CSV
df_corr.to_csv(os.path.join(EDA_DIR, 'correlacoes', '02_matriz_correlacao.csv'))

print(f"✓ Matriz de correlação salva em: correlacoes/")

# Scatter plots das correlações mais fortes
top_correlacoes = []
for i in range(len(df_corr.columns)):
    for j in range(i+1, len(df_corr.columns)):
        top_correlacoes.append({
            'var1': df_corr.columns[i],
            'var2': df_corr.columns[j],
            'correlacao': df_corr.iloc[i, j]
        })

df_top_corr = pd.DataFrame(top_correlacoes)
df_top_corr = df_top_corr.sort_values('correlacao', key=abs, ascending=False).head(6)

# Plotar scatter plots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, row in enumerate(df_top_corr.itertuples()):
    if i < len(axes):
        ax = axes[i]

        var1 = row.var1
        var2 = row.var2
        corr = row.correlacao

        ax.scatter(df[var1], df[var2], alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

        # Linha de tendência
        z = np.polyfit(df[var1].dropna(), df[var2].dropna(), 1)
        p = np.poly1d(z)
        x_line = np.linspace(df[var1].min(), df[var1].max(), 100)
        ax.plot(x_line, p(x_line), 'r--', linewidth=2, label=f'Corr={corr:.3f}')

        ax.set_xlabel(var1, fontsize=10)
        ax.set_ylabel(var2, fontsize=10)
        ax.set_title(f'{var1} × {var2}', fontsize=11, fontweight='bold')
        ax.legend(loc='best', fontsize=9)
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'correlacoes', '03_scatter_top_correlacoes.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Scatter plots salvos em: correlacoes/03_scatter_top_correlacoes.png")

# ==============================================================================
# 12. INDICADORES DERIVADOS
# ==============================================================================
print("\n12. INDICADORES DERIVADOS")
print("-"*100)

# Fator de utilização já existe (capacity_factor_ne)
# Vamos criar outros indicadores

# Excedente/Pressão do Sistema
if 'geracao_eolica_ne_mwmed' in df.columns and 'carga_ne_mwmed' in df.columns:
    df['excedente_sistema'] = df['geracao_eolica_ne_mwmed'] - df['carga_ne_mwmed']
    df['ratio_geracao_carga'] = df['geracao_eolica_ne_mwmed'] / df['carga_ne_mwmed']

# Intensidade de vento (desvio da média)
if 'vel_vento_100m_ne_media' in df.columns:
    media_vento_historica = df['vel_vento_100m_ne_media'].mean()
    df['desvio_vento_percentual'] = ((df['vel_vento_100m_ne_media'] - media_vento_historica) / media_vento_historica) * 100

# Eficiência de conversão (proxy)
if 'geracao_eolica_ne_mwmed' in df.columns and 'potencia_vento_100m_ne' in df.columns:
    df['eficiencia_conversao_proxy'] = df['geracao_eolica_ne_mwmed'] / (df['potencia_vento_100m_ne'] * df['capacidade_eolica_ne_mw'])

# Plotar indicadores
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Excedente do sistema
if 'excedente_sistema' in df.columns:
    axes[0, 0].plot(df['data'], df['excedente_sistema'], linewidth=2)
    axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=1)
    axes[0, 0].fill_between(df['data'], df['excedente_sistema'], 0,
                             where=df['excedente_sistema']>=0, alpha=0.3, color='green', label='Excedente')
    axes[0, 0].fill_between(df['data'], df['excedente_sistema'], 0,
                             where=df['excedente_sistema']<0, alpha=0.3, color='red', label='Déficit')
    axes[0, 0].set_ylabel('MWmed', fontsize=11, fontweight='bold')
    axes[0, 0].set_title('Excedente/Déficit: Geração Eólica - Carga NE', fontsize=12, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

# Ratio Geração/Carga
if 'ratio_geracao_carga' in df.columns:
    axes[0, 1].plot(df['data'], df['ratio_geracao_carga'], linewidth=2, color='purple')
    axes[0, 1].axhline(y=1, color='red', linestyle='--', linewidth=1, label='Equilíbrio')
    axes[0, 1].set_ylabel('Ratio', fontsize=11, fontweight='bold')
    axes[0, 1].set_title('Ratio: Geração Eólica / Carga NE', fontsize=12, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

# Desvio do vento
if 'desvio_vento_percentual' in df.columns:
    axes[1, 0].plot(df['data'], df['desvio_vento_percentual'], linewidth=2, color='darkblue')
    axes[1, 0].axhline(y=0, color='red', linestyle='--', linewidth=1)
    axes[1, 0].fill_between(df['data'], df['desvio_vento_percentual'], 0, alpha=0.3)
    axes[1, 0].set_ylabel('Desvio (%)', fontsize=11, fontweight='bold')
    axes[1, 0].set_title('Desvio da Média Histórica: Velocidade do Vento', fontsize=12, fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3)

# Capacity Factor ao longo do tempo
if 'capacity_factor_ne' in df.columns:
    axes[1, 1].plot(df['data'], df['capacity_factor_ne'], linewidth=2, color='darkgreen')
    axes[1, 1].axhline(y=df['capacity_factor_ne'].mean(), color='red', linestyle='--',
                       linewidth=1, label=f'Média={df["capacity_factor_ne"].mean():.1f}%')
    axes[1, 1].set_ylabel('Capacity Factor (%)', fontsize=11, fontweight='bold')
    axes[1, 1].set_title('Fator de Capacidade Mensal NE', fontsize=12, fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'indicadores', '01_indicadores_derivados.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Indicadores derivados salvos em: indicadores/")

# ==============================================================================
# 13. HIPÓTESES E DRIVERS DE VARIABILIDADE
# ==============================================================================
print("\n13. HIPÓTESES E DRIVERS DE VARIABILIDADE")
print("-"*100)

with open(os.path.join(EDA_DIR, 'hipoteses', '01_analise_drivers.txt'), 'w', encoding='utf-8') as f:
    f.write("="*100 + "\n")
    f.write("ANÁLISE DE DRIVERS DE VARIABILIDADE EÓLICA - NE DO BRASIL\n")
    f.write("="*100 + "\n\n")

    # Hipótese 1: Sazonalidade
    f.write("HIPÓTESE 1: SAZONALIDADE DO VENTO\n")
    f.write("-"*100 + "\n")
    perfil_vento = df.groupby('mes')['vel_vento_100m_ne_media'].mean()
    mes_max_vento = perfil_vento.idxmax()
    mes_min_vento = perfil_vento.idxmin()
    f.write(f"• Mês com maior vento: {meses_nomes[mes_max_vento-1]} ({perfil_vento.max():.2f} m/s)\n")
    f.write(f"• Mês com menor vento: {meses_nomes[mes_min_vento-1]} ({perfil_vento.min():.2f} m/s)\n")
    f.write(f"• Amplitude sazonal: {perfil_vento.max() - perfil_vento.min():.2f} m/s\n")
    f.write(f"• Coeficiente de variação sazonal: {(perfil_vento.std() / perfil_vento.mean() * 100):.2f}%\n\n")

    perfil_cf = df.groupby('mes')['capacity_factor_ne'].mean()
    mes_max_cf = perfil_cf.idxmax()
    mes_min_cf = perfil_cf.idxmin()
    f.write(f"• Mês com maior CF: {meses_nomes[mes_max_cf-1]} ({perfil_cf.max():.2f}%)\n")
    f.write(f"• Mês com menor CF: {meses_nomes[mes_min_cf-1]} ({perfil_cf.min():.2f}%)\n")
    f.write(f"• Amplitude sazonal CF: {perfil_cf.max() - perfil_cf.min():.2f}%\n\n")

    f.write("CONCLUSÃO: Forte sazonalidade observada, com ventos mais intensos no segundo semestre.\n")
    f.write("Isso está alinhado com os padrões climáticos do NE (ventos alísios).\n\n\n")

    # Hipótese 2: Anos anômalos
    f.write("HIPÓTESE 2: ANOS ANÔMALOS\n")
    f.write("-"*100 + "\n")
    cf_por_ano = df.groupby('ano')['capacity_factor_ne'].mean().sort_values(ascending=False)
    f.write("Capacity Factor médio por ano (ranking):\n")
    for i, (ano, cf) in enumerate(cf_por_ano.items(), 1):
        f.write(f"  {i}. {ano}: {cf:.2f}%\n")
    f.write("\n")

    # Identificar anos com CF > +1 std ou < -1 std
    cf_mean = cf_por_ano.mean()
    cf_std = cf_por_ano.std()
    anos_alto = cf_por_ano[cf_por_ano > cf_mean + cf_std]
    anos_baixo = cf_por_ano[cf_por_ano < cf_mean - cf_std]

    if len(anos_alto) > 0:
        f.write(f"Anos com CF ACIMA DA MÉDIA (+1σ):\n")
        for ano, cf in anos_alto.items():
            f.write(f"  • {ano}: {cf:.2f}% (desvio: +{cf-cf_mean:.2f}%)\n")
        f.write("\n")

    if len(anos_baixo) > 0:
        f.write(f"Anos com CF ABAIXO DA MÉDIA (-1σ):\n")
        for ano, cf in anos_baixo.items():
            f.write(f"  • {ano}: {cf:.2f}% (desvio: {cf-cf_mean:.2f}%)\n")
        f.write("\n")

    f.write("CONCLUSÃO: Variabilidade interanual significativa, possivelmente associada a\n")
    f.write("fenômenos climáticos de larga escala (El Niño, La Niña, etc.).\n\n\n")

    # Hipótese 3: Curtailment
    f.write("HIPÓTESE 3: CURTAILMENT E RESTRIÇÕES DE ESCOAMENTO\n")
    f.write("-"*100 + "\n")
    curtailment_medio = df['corte_eolica_ne_mwmed'].mean()
    curtailment_max = df['corte_eolica_ne_mwmed'].max()
    idx_curt_max = df['corte_eolica_ne_mwmed'].idxmax()

    f.write(f"• Curtailment médio: {curtailment_medio:.2f} MWmed\n")
    f.write(f"• Curtailment máximo: {curtailment_max:.2f} MWmed (em {df.loc[idx_curt_max, 'data'].strftime('%Y-%m')})\n")
    f.write(f"• Meses com curtailment > 100 MWmed: {(df['corte_eolica_ne_mwmed'] > 100).sum()} meses\n\n")

    # Correlação entre penetração e curtailment
    if 'penetracao_eolica_ne' in df.columns:
        corr_pen_curt = df[['penetracao_eolica_ne', 'corte_eolica_ne_mwmed']].corr().iloc[0, 1]
        f.write(f"• Correlação penetração × curtailment: {corr_pen_curt:.3f}\n")
        f.write("CONCLUSÃO: Curtailment aumenta com penetração eólica, sugerindo restrições\n")
        f.write("de escoamento e necessidade de expansão da transmissão.\n\n\n")

    # Hipótese 4: Crescimento da capacidade
    f.write("HIPÓTESE 4: IMPACTO DO CRESCIMENTO DA CAPACIDADE\n")
    f.write("-"*100 + "\n")
    cap_inicial = df['capacidade_eolica_ne_mw'].iloc[0]
    cap_final = df['capacidade_eolica_ne_mw'].iloc[-1]
    crescimento_total = ((cap_final - cap_inicial) / cap_inicial) * 100

    f.write(f"• Capacidade inicial ({df['data'].iloc[0].strftime('%Y-%m')}): {cap_inicial:.0f} MW\n")
    f.write(f"• Capacidade final ({df['data'].iloc[-1].strftime('%Y-%m')}): {cap_final:.0f} MW\n")
    f.write(f"• Crescimento total: {crescimento_total:.1f}%\n")
    f.write(f"• CAGR: {df_tendencias[df_tendencias['variavel']=='capacidade_eolica_ne_mw']['cagr_%'].values[0]:.2f}%/ano\n\n")

    f.write("CONCLUSÃO: Crescimento acelerado da capacidade, com potencial impacto na\n")
    f.write("confiabilidade do sistema e necessidade de flexibilidade operacional.\n\n")

print(f"✓ Análise de drivers salva em: hipoteses/01_analise_drivers.txt")

# Gráfico final: Evolução conjunta dos principais drivers
fig, axes = plt.subplots(4, 1, figsize=(16, 14), sharex=True)

# Capacidade e Geração
ax1 = axes[0]
ax1_twin = ax1.twinx()
ax1.plot(df['data'], df['capacidade_eolica_ne_mw'], 'b-', linewidth=2, label='Capacidade (MW)')
ax1_twin.plot(df['data'], df['geracao_eolica_ne_mwmed'], 'g-', linewidth=2, label='Geração (MWmed)')
ax1.set_ylabel('Capacidade (MW)', fontsize=11, fontweight='bold', color='b')
ax1_twin.set_ylabel('Geração (MWmed)', fontsize=11, fontweight='bold', color='g')
ax1.set_title('Evolução Conjunta: Capacidade vs Geração', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper left')
ax1_twin.legend(loc='upper right')

# Vento e CF
ax2 = axes[1]
ax2_twin = ax2.twinx()
ax2.plot(df['data'], df['vel_vento_100m_ne_media'], 'darkblue', linewidth=2, label='Vento 100m (m/s)')
ax2_twin.plot(df['data'], df['capacity_factor_ne'], 'orange', linewidth=2, label='Capacity Factor (%)')
ax2.set_ylabel('Vento (m/s)', fontsize=11, fontweight='bold', color='darkblue')
ax2_twin.set_ylabel('CF (%)', fontsize=11, fontweight='bold', color='orange')
ax2.set_title('Evolução Conjunta: Vento vs Capacity Factor', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper left')
ax2_twin.legend(loc='upper right')

# Penetração e Curtailment
ax3 = axes[2]
ax3_twin = ax3.twinx()
ax3.plot(df['data'], df['penetracao_eolica_ne'], 'purple', linewidth=2, label='Penetração NE (%)')
ax3_twin.plot(df['data'], df['corte_eolica_ne_mwmed'], 'red', linewidth=2, label='Curtailment (MWmed)')
ax3.set_ylabel('Penetração (%)', fontsize=11, fontweight='bold', color='purple')
ax3_twin.set_ylabel('Curtailment (MWmed)', fontsize=11, fontweight='bold', color='red')
ax3.set_title('Evolução Conjunta: Penetração vs Curtailment', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.legend(loc='upper left')
ax3_twin.legend(loc='upper right')

# Demanda NE vs SIN
ax4 = axes[3]
ax4_twin = ax4.twinx()
ax4.plot(df['data'], df['carga_ne_mwmed'], 'teal', linewidth=2, label='Carga NE (MWmed)')
ax4_twin.plot(df['data'], df['demanda_sin_mwmed'], 'brown', linewidth=2, label='Demanda SIN (MWmed)')
ax4.set_xlabel('Data', fontsize=12, fontweight='bold')
ax4.set_ylabel('Carga NE (MWmed)', fontsize=11, fontweight='bold', color='teal')
ax4_twin.set_ylabel('Demanda SIN (MWmed)', fontsize=11, fontweight='bold', color='brown')
ax4.set_title('Evolução Conjunta: Carga NE vs Demanda SIN', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.legend(loc='upper left')
ax4_twin.legend(loc='upper right')

plt.tight_layout()
plt.savefig(os.path.join(EDA_DIR, 'hipoteses', '02_evolucao_conjunta_drivers.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Gráfico de evolução conjunta salvo em: hipoteses/02_evolucao_conjunta_drivers.png")

# ==============================================================================
# RELATÓRIO FINAL
# ==============================================================================
print("\n" + "="*100)
print("EDA COMPLETA FINALIZADA COM SUCESSO!")
print("="*100)
print(f"\nResumo dos outputs gerados:")
print(f"  • Diretório principal: {EDA_DIR}")
print(f"  • Subdiretórios: {len(subdirs)}")
print(f"  • Gráficos gerados: ~40+")
print(f"  • Tabelas CSV: ~10+")
print(f"  • Relatórios TXT: 3")
print(f"\nPróximas etapas recomendadas:")
print(f"  1. Revisar relatórios de validação e integridade")
print(f"  2. Analisar séries temporais e identificar anomalias")
print(f"  3. Investigar correlações fortes entre variáveis")
print(f"  4. Validar hipóteses sobre drivers de variabilidade")
print(f"  5. Preparar features para modelagem com base nas análises")
print("="*100)


EDA COMPLETA - DATASET FINAL DE MODELAGEM

1. CARREGAMENTO E VALIDAÇÃO DO DICIONÁRIO DE DADOS
----------------------------------------------------------------------------------------------------
✓ Dataset carregado: 132 linhas × 2265 colunas
✓ Dicionário de dados salvo em: validacao/00_dicionario_dados.txt

2. CONSISTÊNCIA E INTEGRIDADE
----------------------------------------------------------------------------------------------------
✓ Sequência temporal: 0 meses faltantes
✓ Duplicatas: 0 registros duplicados
✓ Valores faltantes: 72864 células (24.37%)
✓ Relatório de outliers salvo em: validacao/01_outliers_report.csv
✓ Relatório de integridade salvo em: validacao/02_integridade.txt

3. SÉRIES TEMPORAIS PRINCIPAIS
----------------------------------------------------------------------------------------------------
✓ 8 séries temporais salvas em: series_temporais/

4. ESTATÍSTICAS DESCRITIVAS COMPLETAS
------------------------------------------------------------------------------------

KeyError: "Columns not found: 'temp_celsius_ne', 'vel_vento_100m_ne_media'"

## Pipeline dos Modelos

In [ ]:
#Pipeline do Ensemble

In [ ]:
#Pipeline do XGBoost

## Estatisticas Finais